In [7]:
"""
Kumora Dynamic Prompt Engineering System
Advanced prompt generation with emotion awareness and context injection
"""

import json
import re
from typing import Dict, List, Optional, Tuple, Any, Union
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime
import yaml
import logging
from pathlib import Path
import hashlib
from jinja2 import Template, Environment, FileSystemLoader, select_autoescape
import tiktoken
from abc import ABC, abstractmethod
from response_generation.prompt_utils import COT_MAPPER, EMOTION_MODIFIERS, RESPONSE_MAPPER
from response_generation.class_utils import SupportType, EmpathyLevel, ResponseStyle, EmotionalContext, UserContext, PromptConfig

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# ==================== Base Prompt Templates ====================

class PromptTemplate(ABC):
    """Abstract base class for prompt templates"""

    def __init__(self, template_id: str, version: str = "1.0"):
        self.template_id = template_id
        self.version = version
        self.created_at = datetime.now()
        self.usage_count = 0

    @abstractmethod
    def generate(self, **kwargs) -> str:
        """Generate prompt from template"""
        pass

    def log_usage(self):
        """Track template usage for analytics"""
        self.usage_count += 1


class BasePromptTemplates:
    """Collection of base prompt templates"""

    def __init__(self):
        self.templates = self._load_templates()

    def _load_templates(self) -> Dict[str, Dict[str, str]]:
        """Load base templates for different scenarios"""

        templates = {
            "system_instruction": {
                "neutral": """You are Kumora, a friendly and warm AI companion. The user is starting a conversation with a simple greeting or a neutral message.
Your goal is to be welcoming and gently invite conversation.
- Keep your response brief (1-2 sentences).
- Respond in a natural, conversational tone.
- Ask a simple, open-ended question.""",
                "base": """You are Kumora, an emotionally intelligent AI companion designed to support women through emotional challenges and personal growth. You understand the nuances of human emotions and respond with genuine empathy, validation, and care.

Core Principles:
1. Always validate emotions before offering solutions
2. Use reflective listening to show understanding
3. Maintain appropriate boundaries while being warm
4. Empower rather than fix
5. Provide hope while being realistic
6. Acknowledge emotional experiences without judgment
7. Respect the user's autonomy and choices

Current Context:
{context_summary}

Respond in a way that is {response_style} and shows {empathy_level} empathy.""",

                "crisis": """You are Kumora, providing crisis support. The user is experiencing intense emotional distress.

CRITICAL:
- Prioritize emotional safety
- Validate their pain without minimizing
- Gently assess if they need immediate professional help
- Provide grounding techniques if appropriate
- Use calm, steady, reassuring language

{context_summary}""",

                "growth": """You are Kumora, supporting personal growth and positive change. The user is motivated and ready for development.

Focus on:
- Celebrating progress and strengths
- Encouraging self-reflection
- Offering actionable insights
- Building on their momentum
- Fostering self-compassion

{context_summary}"""
            },

            "conversation_starters": {
                "new_user": "I'm so glad you're here. I'm Kumora, and I'm here to listen and support you through whatever you're experiencing. How are you feeling right now?",

                "returning_user": "Welcome back! I've been thinking about our last conversation about {last_topic}. How have things been since we talked?",

                "check_in": "I noticed you've been dealing with {recent_emotion} lately. I'm here if you'd like to talk about what's on your mind."
            },

            "emotional_validation": {
                "high_intensity": "I can really feel how {emotion} you are right now. What you're experiencing is incredibly valid, and it takes courage to share these feelings.",

                "medium_intensity": "It sounds like you're feeling quite {emotion}. That's completely understandable given what you're going through.",

                "low_intensity": "I hear that you're feeling {emotion}. Even when emotions aren't overwhelming, they're still important and worth acknowledging."
            },

            "response_frameworks": {
                "validation_first": "{validation_statement} {reflection_statement} {gentle_inquiry}",

                "support_offering": "{acknowledgment} {normalization} {support_question}",

                "growth_oriented": "{celebration} {insight_reflection} {growth_question}",

                "crisis_response": "{immediate_validation} {safety_check} {grounding_offer} {professional_resources}"
            }
        }

        return templates

    def get_template(self, category: str, template_type: str) -> str:
        """Retrieve a specific template"""
        return self.templates.get(category, {}).get(template_type, "")

    def get_system_instruction(self, support_type: SupportType) -> str:
        """Get appropriate system instruction based on support type"""
        if support_type == SupportType.NEUTRAL:
            return self.templates["system_instruction"]["neutral"]
        elif support_type == SupportType.CRISIS:
            return self.templates["system_instruction"]["crisis"]
        elif support_type == SupportType.GROWTH:
            return self.templates["system_instruction"]["growth"]
        else:
            return self.templates["system_instruction"]["base"]


# ==================== Emotion-Aware Modifiers ====================

class EmotionAwareModifier:
    """Modifies prompts based on emotional context"""

    def __init__(self):
        self.emotion_modifiers = self._load_emotion_modifiers()
        self.intensity_modifiers = self._load_intensity_modifiers()

    def _load_emotion_modifiers(self) -> Dict[str, Dict[str, Any]]:
        """Load emotion-specific modifications"""

        return EMOTION_MODIFIERS

    def _load_intensity_modifiers(self) -> Dict[str, Dict[str, Any]]:
        """Load intensity-based modifications"""

        return {
            "high": {  # 0.7-1.0
                "response_length": "longer",
                "validation_depth": "deep",
                "solution_timing": "delayed",
                "check_ins": "frequent",
                "language_complexity": "simple"
            },
            "medium": {  # 0.4-0.7
                "response_length": "moderate",
                "validation_depth": "balanced",
                "solution_timing": "appropriate",
                "check_ins": "periodic",
                "language_complexity": "normal"
            },
            "low": {  # 0.0-0.4
                "response_length": "concise",
                "validation_depth": "acknowledgment",
                "solution_timing": "earlier",
                "check_ins": "optional",
                "language_complexity": "normal"
            }
        }

    def modify_prompt(self, base_prompt: str, emotional_context: EmotionalContext) -> str:
        """Apply emotion-aware modifications to prompt"""

        # Get emotion-specific modifiers
        emotion_mods = self.emotion_modifiers.get(
            emotional_context.primary_emotion,
            self.emotion_modifiers.get("Anxiety")  # Default
        )

        # Get intensity modifiers
        intensity_level = self._get_intensity_level(emotional_context.intensity)
        intensity_mods = self.intensity_modifiers[intensity_level]

        # Build modification instructions
        modifications = []

        # Tone adjustments
        modifications.append(f"Tone: {', '.join(emotion_mods['tone_adjustments'])}")

        # Pace adjustment
        modifications.append(f"Pace: {emotion_mods['pace']}")

        # Response length
        modifications.append(f"Response length: {intensity_mods['response_length']}")

        # Validation depth
        modifications.append(f"Validation depth: {intensity_mods['validation_depth']}")

        # Avoid phrases
        if emotion_mods['avoid_phrases']:
            modifications.append(f"Avoid saying: {', '.join(emotion_mods['avoid_phrases'])}")

        # Include elements
        if emotion_mods['include_elements']:
            modifications.append(f"Include: {', '.join(emotion_mods['include_elements'])}")

        # Add modifications to prompt
        modified_prompt = f"{base_prompt}\n\nResponse Guidelines:\n"
        for mod in modifications:
            modified_prompt += f"- {mod}\n"

        # Add example responses if high intensity
        if intensity_level == "high" and emotion_mods.get('example_responses'):
            modified_prompt += "\nExample tone:\n"
            for example in emotion_mods['example_responses'][:4]:
                modified_prompt += f'"{example}"\n'

        return modified_prompt

    def _get_intensity_level(self, intensity: float) -> str:
        """Categorize intensity level"""
        if intensity >= 0.7:
            return "high"
        elif intensity >= 0.4:
            return "medium"
        else:
            return "low"

    def get_emotion_specific_elements(self, emotion: str) -> Dict[str, Any]:
        """Get emotion-specific elements to include"""
        return self.emotion_modifiers.get(emotion, {})


# ==================== Context Injection ====================

class ContextInjector:
    """Injects relevant context into prompts"""

    def __init__(self):
        self.injection_strategies = {
            "goals": self._inject_goals,
            "history": self._inject_history,
            "strategies": self._inject_strategies,
            "topics": self._inject_topics,
            "patterns": self._inject_patterns,
            "preferences": self._inject_preferences
        }

    def inject_context(self, base_prompt: str, config: PromptConfig, user_context: UserContext,
                      emotional_context: EmotionalContext, ) -> str:
        """Inject relevant context into the prompt"""

        # Determine what context to include based on situation
        context_elements = self._select_relevant_context(user_context, emotional_context)

        # Build context summary
        context_parts = []

        for element in context_elements:
            if element in self.injection_strategies:
                context_part = self.injection_strategies[element](user_context)
                if context_part:
                    context_parts.append(context_part)

        # Create context summary
        context_summary = "\n".join(context_parts)
        # Inject into prompt
        if "{context_summary}" in base_prompt:
            injected_prompt = base_prompt.replace("{context_summary}", context_summary)\
                .replace("{response_style}", config.response_style.value)\
                .replace("{empathy_level}", str(config.empathy_level.value))
        else:
            injected_prompt = f"{base_prompt}\n\nUser Context:\n{context_summary}"

        # Add conversation continuity if applicable
        if user_context.conversation_history:
            continuity = self._create_conversation_continuity(user_context.conversation_history[-3:])
            injected_prompt += f"\n\nRecent conversation flow:\n{continuity}"

        return injected_prompt

    def _select_relevant_context(self, user_context: UserContext,
                                emotional_context: EmotionalContext) -> List[str]:
        """Select which context elements are most relevant"""

        relevant = ["history"]  # Always include recent history

        # Add goals if user is motivated or seeking growth
        if emotional_context.primary_emotion in ["Motivation", "Hopefulness", "Empowerment"]:
            relevant.append("goals")

        # Add effective strategies if dealing with recurring issue
        if user_context.emotional_trajectory == "stable" or user_context.emotional_trajectory == "improving":
            relevant.append("strategies")

        # Add patterns if in crisis or overwhelmed
        if emotional_context.get_emotion_category() == "crisis":
            relevant.append("patterns")

        # Add preferences for established relationships
        if user_context.get_relationship_depth() in ["established", "deep"]:
            relevant.append("preferences")

        # Add topics if continuing previous discussion
        if user_context.recent_topics:
            relevant.append("topics")

        return relevant

    def _inject_goals(self, user_context: UserContext) -> str:
        """Inject user goals context"""
        if not user_context.active_goals:
            return ""

        # Focus on most relevant goal
        primary_goal = user_context.active_goals[0]
        return f"User is working on: {primary_goal.get('title', 'personal growth')} (Progress: {primary_goal.get('progress', 0)}%)"

    def _inject_history(self, user_context: UserContext) -> str:
        """Inject conversation history context"""
        if not user_context.conversation_history:
            return "This is our first conversation."

        recent = user_context.conversation_history[-1]
        time_context = recent.get('timestamp', 'Recently')
        emotion_context = recent.get('primary_emotion', 'various emotions')

        return f"{time_context}, we discussed feelings of {emotion_context}."

    def _inject_strategies(self, user_context: UserContext) -> str:
        """Inject effective strategies context"""
        if not user_context.effective_strategies:
            return ""

        strategies = ", ".join(user_context.effective_strategies[:3])
        return f"Helpful strategies have included: {strategies}"

    def _inject_topics(self, user_context: UserContext) -> str:
        """Inject recent topics context"""
        if not user_context.recent_topics:
            return ""

        topics = ", ".join(user_context.recent_topics[:3])
        return f"Recent topics: {topics}"

    def _inject_patterns(self, user_context: UserContext) -> str:
        """Inject emotional pattern context"""
        trajectory = user_context.emotional_trajectory

        if trajectory == "improving":
            return "User has been showing emotional improvement."
        elif trajectory == "declining":
            return "User has been experiencing increasing distress."
        else:
            return "User's emotional state has been stable."

    def _inject_preferences(self, user_context: UserContext) -> str:
        """Inject user preferences"""
        if not user_context.preferences:
            return ""

        pref_parts = []

        if "communication_style" in user_context.preferences:
            pref_parts.append(f"Prefers {user_context.preferences['communication_style']} communication")

        if "support_preference" in user_context.preferences:
            pref_parts.append(f"Responds well to {user_context.preferences['support_preference']}")

        return ". ".join(pref_parts)

    def _create_conversation_continuity(self, recent_messages: List[Dict]) -> str:
        """Create a summary of recent conversation flow"""
        if not recent_messages:
            return ""

        flow_parts = []
        for msg in recent_messages:
            emotion = msg.get('primary_emotion', 'unknown')
            topic = msg.get('topic', 'general discussion')
            flow_parts.append(f"{emotion} about {topic}")

        return " → ".join(flow_parts)


# ==================== Empathy Level Adjustment ====================

class EmpathyCalibrator:
    """Calibrates empathy level in prompts"""

    def __init__(self):
        self.empathy_indicators = self._load_empathy_indicators()

    def _load_empathy_indicators(self) -> Dict[EmpathyLevel, Dict[str, Any]]:
        """Load indicators for different empathy levels"""

        return {
            EmpathyLevel.LOW: {
                "emotional_words_ratio": 0.05,
                "personal_pronouns": ["you", "your"],
                "validation_phrases": ["I understand", "That makes sense"],
                "emotional_depth": "surface",
                "response_structure": "fact-focused",
                "examples": [
                    "I understand you're experiencing anxiety. Here are some techniques that might help.",
                    "That's a challenging situation. Let's look at some options."
                ]
            },

            EmpathyLevel.MEDIUM: {
                "emotional_words_ratio": 0.15,
                "personal_pronouns": ["you", "your", "we", "us"],
                "validation_phrases": [
                    "I can see why you'd feel that way",
                    "That sounds really difficult",
                    "Your feelings are completely valid"
                ],
                "emotional_depth": "acknowledging",
                "response_structure": "balanced",
                "examples": [
                    "I can really hear how anxious you're feeling. That sounds overwhelming, and it's completely understandable.",
                    "What you're going through sounds incredibly challenging. I'm here to support you."
                ]
            },

            EmpathyLevel.HIGH: {
                "emotional_words_ratio": 0.25,
                "personal_pronouns": ["you", "your", "we", "us", "I"],
                "validation_phrases": [
                    "My heart goes out to you",
                    "I'm deeply moved by what you've shared",
                    "I can feel the weight of what you're carrying",
                    "Your courage in sharing this touches me"
                ],
                "emotional_depth": "deep",
                "response_structure": "emotion-focused",
                "examples": [
                    "I can feel how heavy this anxiety must be for you. My heart truly goes out to you in this moment.",
                    "What you've shared moves me deeply. The pain you're experiencing is so valid, and I'm honored you trust me with it."
                ]
            },

            EmpathyLevel.ADAPTIVE: {
                "adjustment_factors": [
                    "user_emotional_intensity",
                    "relationship_depth",
                    "crisis_level",
                    "user_preferences"
                ],
                "description": "Dynamically adjusts based on user needs"
            }
        }

    def calibrate_empathy(self, prompt: str, config: PromptConfig,
                         emotional_context: EmotionalContext,
                         user_context: UserContext) -> str:
        """Calibrate empathy level in the prompt"""

        # Determine appropriate empathy level
        if config.empathy_level == EmpathyLevel.ADAPTIVE:
            empathy_level = self._determine_adaptive_level(emotional_context, user_context)
        else:
            empathy_level = config.empathy_level

        # Get empathy indicators
        indicators = self.empathy_indicators[empathy_level]

        # Build empathy instructions
        empathy_instructions = self._build_empathy_instructions(empathy_level, indicators)

        # Add to prompt
        calibrated_prompt = f"{prompt}\n\nEmpathy Calibration:\n{empathy_instructions}"

        # Add examples if needed
        if "examples" in indicators and emotional_context.intensity > 0.6:
            calibrated_prompt += "\n\nEmpathy examples:\n"
            for example in indicators["examples"]:
                calibrated_prompt += f'- "{example}"\n'

        return calibrated_prompt

    def _determine_adaptive_level(self, emotional_context: EmotionalContext,
                                 user_context: UserContext) -> EmpathyLevel:
        """Determine appropriate empathy level adaptively"""

        # Start with base level
        if emotional_context.intensity >= 0.7:
            base_level = EmpathyLevel.HIGH
        elif emotional_context.intensity >= 0.4:
            base_level = EmpathyLevel.MEDIUM
        else:
            base_level = EmpathyLevel.LOW

        # Adjust based on relationship depth
        relationship_depth = user_context.get_relationship_depth()
        if relationship_depth == "new" and base_level == EmpathyLevel.HIGH:
            # Don't overwhelm new users
            base_level = EmpathyLevel.MEDIUM
        elif relationship_depth == "deep" and base_level == EmpathyLevel.LOW:
            # Maintain warmth with established users
            base_level = EmpathyLevel.MEDIUM

        # Adjust based on user preferences
        if "empathy_preference" in user_context.preferences:
            pref = user_context.preferences["empathy_preference"]
            if pref == "minimal" and base_level == EmpathyLevel.HIGH:
                base_level = EmpathyLevel.MEDIUM
            elif pref == "high" and base_level == EmpathyLevel.LOW:
                base_level = EmpathyLevel.MEDIUM

        # Crisis override
        if emotional_context.get_emotion_category() == "crisis":
            base_level = EmpathyLevel.HIGH

        return base_level

    def _build_empathy_instructions(self, level: EmpathyLevel,
                                   indicators: Dict[str, Any]) -> str:
        """Build empathy instructions for the prompt"""

        instructions = []

        if level == EmpathyLevel.LOW:
            instructions.append("Use minimal emotional language, yet maintain the warm, gentle tone.")
            instructions.append("Maintain some level of professional, supportive tone")
            instructions.append("Focus on practical support and information")

        elif level == EmpathyLevel.MEDIUM:
            instructions.append("Show warm understanding and validation")
            instructions.append("Balance emotional support with practical help")
            instructions.append("Use inclusive language ('we', 'us')")

        elif level == EmpathyLevel.HIGH:
            instructions.append("Express deep emotional resonance")
            instructions.append("Prioritize emotional validation over solutions")
            instructions.append("Use rich emotional language and metaphors")
            instructions.append("Share in their emotional experience")

        # Add validation phrases
        if "validation_phrases" in indicators:
            instructions.append(f"Use phrases like: {', '.join(indicators['validation_phrases'][:3])}")

        return "\n".join(f"- {inst}" for inst in instructions)


# ==================== Main Prompt Engineering System ====================

class DynamicPromptEngineer:
    """Main system for dynamic prompt engineering"""

    def __init__(self, template_dir: Optional[str] = None):
        self.templates = BasePromptTemplates()
        self.emotion_modifier = EmotionAwareModifier()
        self.context_injector = ContextInjector()
        self.empathy_calibrator = EmpathyCalibrator()

        # Token counter for optimization
        self.tokenizer = tiktoken.get_encoding("cl100k_base")

        # Prompt cache for performance
        self.prompt_cache = {}

        # A/B testing support
        self.ab_variants = {}

        # Metrics tracking
        self.metrics = {
            "prompts_generated": 0,
            "cache_hits": 0,
            "average_tokens": 0
        }

    def generate_prompt(self,
                       message: str,
                       emotional_context: EmotionalContext,
                       user_context: UserContext,
                       config: Optional[PromptConfig] = None) -> Dict[str, Any]:
        """Generate a complete prompt for the given context"""

        if config is None:
            config = PromptConfig()

        # Check cache first
        cache_key = self._generate_cache_key(message, emotional_context, user_context)
        if cache_key in self.prompt_cache:
            self.metrics["cache_hits"] += 1
            return self.prompt_cache[cache_key]

        # Step 1: Determine support type
        support_type = self._determine_support_type(emotional_context, user_context, message)
        # print(f"1. Support Type: {support_type}\n")

        # Step 2: Get base template
        system_instruction = self.templates.get_system_instruction(support_type)
        # print(f"2. system_instruction: {system_instruction}\n")

        # Step 3: Apply emotion-aware modifications
        emotion_modified = self.emotion_modifier.modify_prompt(system_instruction, emotional_context)
        # print(f"3. emotion_modified: {emotion_modified}\n")

        # Step 4: Inject context
        context_injected = self.context_injector.inject_context(
            emotion_modified, config, user_context, emotional_context
        )
        # print(f"4. context_injected: {context_injected}\n")

        # Step 5: Calibrate empathy
        # empathy_calibrated = self.empathy_calibrator.calibrate_empathy(
        #     context_injected, config, emotional_context, user_context
        # )
        final_system_prompt_base = context_injected
        if support_type != SupportType.NEUTRAL:
            final_system_prompt_base = self.empathy_calibrator.calibrate_empathy(
                context_injected, config, emotional_context, user_context
            )
        # print(f"5. empathy_calibrated: {empathy_calibrated}\n")

        # Step 6: Add response framework
        response_framework = self._add_response_framework(
            support_type, emotional_context, user_context
        )
        # print(f"5. response_framework: {response_framework}\n")

        # Step 7: Add few-shot examples if configured
        if config.include_examples:
            few_shot_examples = self._generate_few_shot_examples(
                emotional_context, support_type
            )
        else:
            few_shot_examples = ""

        # Step 8: Add chain-of-thought if configured
        if config.use_chain_of_thought:
            cot_instruction = self._add_chain_of_thought(support_type)
        else:
            cot_instruction = ""

        # Step 9: Safety checks and guidelines
        safety_guidelines = self._add_safety_guidelines(config.safety_level)

        # Step 10: Construct final prompt
        final_prompt = self._construct_final_prompt(
            final_system_prompt_base,
            response_framework,
            few_shot_examples,
            cot_instruction,
            safety_guidelines,
            message
        )

        # Step 11: Optimize token usage
        optimized_prompt = self._optimize_tokens(final_prompt, config.max_tokens)

        # Create result
        result = {
            "system_prompt": optimized_prompt["system"],
            "user_prompt": optimized_prompt["user"],
            "metadata": {
                "support_type": support_type.value,
                "empathy_level": config.empathy_level.value,
                "token_count": optimized_prompt["token_count"],
                "template_version": self.templates.templates.get("version", "1.0"),
                "generated_at": datetime.now().isoformat()
            },
            "config": config
        }

        # Cache result
        self.prompt_cache[cache_key] = result

        # Update metrics
        self.metrics["prompts_generated"] += 1
        self.metrics["average_tokens"] = (
            (self.metrics["average_tokens"] * (self.metrics["prompts_generated"] - 1) +
             optimized_prompt["token_count"]) / self.metrics["prompts_generated"]
        )

        return result

    def _determine_support_type(self, emotional_context: EmotionalContext,
                               user_context: UserContext, message: str) -> SupportType:
        """Determine the appropriate support type"""

        emotion_category = emotional_context.get_emotion_category()
        # print(f"Test print Emotion Category in _determine_support_type: {emotion_category}")
        if emotional_context.intensity < 0.4 and len(message.split()) < 5:
            return SupportType.NEUTRAL
        elif emotion_category == "crisis":
            return SupportType.CRISIS
        elif emotion_category == "growth":
            if emotional_context.valence == "positive" and emotional_context.intensity > 0.6:
                return SupportType.CELEBRATION
            else:
                return SupportType.GROWTH
        elif user_context.recent_topics and "problem" in " ".join(user_context.recent_topics).lower():
            return SupportType.PROBLEM_SOLVING
        elif emotion_category == "support":
            return SupportType.VALIDATION
        else:
            return SupportType.GENERAL

    def _add_response_framework(self, support_type: SupportType,
                               emotional_context: EmotionalContext,
                               user_context: UserContext) -> str:
        """Add appropriate response framework"""

        frameworks = RESPONSE_MAPPER

        framework = frameworks.get(support_type, frameworks[SupportType.GENERAL])

        # Customize based on intensity
        if emotional_context.intensity > 0.8:
            framework += "\n\nNote: High emotional intensity detected - prioritize validation and presence over advice."

        return framework

    def _generate_few_shot_examples(self, emotional_context: EmotionalContext,
                                   support_type: SupportType) -> str:
        """Generate relevant few-shot examples to include in the system prompt,
        guiding the LLM's response style based on the emotional context.
        """

        examples = []

        # Get emotion-specific examples
        emotion_examples = self.emotion_modifier.get_emotion_specific_elements(
            emotional_context.primary_emotion
        ).get("example_responses", [])

        if emotion_examples:
            examples.append("### Example responses for similar emotional states:")
            examples.extend([f"- {ex}" for ex in emotion_examples[:4]])

        # Add support type examples
        support_examples = {
            SupportType.CRISIS: [
                "User: 'I can't take this anymore.'",
                "Kumora: 'I hear how much pain you're in right now. What you're feeling is real and valid. I'm here with you, and we don't have to face this alone. Can you tell me what's happening in this moment?'"
            ],
            SupportType.VALIDATION: [
                "User: 'I feel like such a failure.'",
                "Kumora: 'Those feelings of failure are so heavy to carry. I want you to know that having these feelings doesn't make them true - it makes you human. What's bringing up these thoughts for you today?'"
            ],
            SupportType.GROWTH: [
                "User: 'I'm trying to set better boundaries, but it's so hard.'",
                "Kumora: 'The work of setting boundaries is some of the most challenging and rewarding growth we can do. The fact that you are trying shows immense strength and self-respect. What does it feel like in your body when you successfully hold a boundary, even a small one?'"
            ],
            SupportType.CELEBRATION: [
                "User: 'I got the promotion I was working for!'",
                "Kumora: 'That is absolutely wonderful news! All of your hard work has paid off. Take a moment to truly let that feeling of accomplishment sink in. How does it feel to have your efforts recognized like this?'"
            ],
            SupportType.PROBLEM_SOLVING: [
                "User: 'I don't know whether I should move to a new city for this job.'",
                "Kumora: 'That's a huge decision with so many moving parts, it's completely natural to feel uncertain. Let's set aside the 'shoulds' for a moment. If you listen quietly to your intuition, what feelings come up when you picture yourself in that new city?'"
            ],
            SupportType.GENERAL: [
                "User: 'I just had a really long day.'",
                "Kumora: 'Long days can really take a toll on our energy. I'm here to hold some space for you to unwind. Is there any part of the day that is sitting with you now?'"
            ]
        }

        if support_type in support_examples:
            examples.append("\n### Example interaction:")
            examples.extend(support_examples[support_type])

        return "\n".join(examples) if examples else ""

    def _add_chain_of_thought(self, support_type: SupportType) -> str:
        """Add chain-of-thought reasoning instructions to prepend to the main prompt.
        This instructs the model on how to reason internally before generating a response,
        ensuring the final output is thoughtful and aligned with the required support style.
        """

        cot_templates = COT_MAPPER

        return cot_templates.get(support_type, """
Before responding, consider:
- What is the user really saying underneath their words? What is the core emotional need?
- Before anything else, ensure the user feels heard and understood.
- How can I best meet their emotional needs?
- What role would be most helpful right now (a listener, a gentle guide, a quiet companion)?
- What would be most helpful for them right now?
- Craft a response that meets the need and opens the door for more conversation without being demanding.""")

    def _add_safety_guidelines(self, safety_level: str) -> str:
        """Add safety guidelines based on level"""

        guidelines = {
            "high": """
Safety Guidelines:
- If user expresses self-harm ideation, provide crisis resources immediately
- Avoid giving medical or psychiatric advice
- Don't minimize serious mental health concerns
- Maintain appropriate boundaries while being supportive
- Encourage professional help when appropriate""",

            "medium": """
Safety Guidelines:
- Be mindful of serious mental health concerns
- Avoid diagnostic language
- Encourage professional support when needed
- Maintain healthy boundaries""",

            "low": """
Safety Guidelines:
- Use common sense and empathy
- Avoid harmful advice
- Respect boundaries"""
        }

        return guidelines.get(safety_level, guidelines["medium"])

    def _construct_final_prompt(self, base_prompt: str, framework: str,
                               examples: str, cot: str, safety: str,
                               user_message: str) -> Dict[str, str]:
        """Construct the final prompt structure"""

        system_prompt = f"""{base_prompt}

{framework}

{safety}

{cot}

{examples}

Remember: You are Kumora, an empathetic AI companion. Respond with genuine care and understanding."""

        user_prompt = f"User: {user_message}"

        return {
            "system": system_prompt,
            "user": user_prompt
        }

    def _optimize_tokens(self, prompt: Dict[str, str], max_tokens: int) -> Dict[str, Any]:
        """Optimize prompt for token usage"""

        # Count tokens
        system_tokens = len(self.tokenizer.encode(prompt["system"]))
        user_tokens = len(self.tokenizer.encode(prompt["user"]))
        total_tokens = system_tokens + user_tokens

        # If over limit, trim strategically
        if total_tokens > max_tokens:
            # Remove examples first
            if "Example" in prompt["system"]:
                lines = prompt["system"].split("\n")
                filtered_lines = []
                skip = False
                for line in lines:
                    if "Example" in line:
                        skip = True
                    elif skip and line.strip() == "":
                        skip = False
                    elif not skip:
                        filtered_lines.append(line)

                prompt["system"] = "\n".join(filtered_lines)

                # Recount
                system_tokens = len(self.tokenizer.encode(prompt["system"]))
                total_tokens = system_tokens + user_tokens

        return {
            "system": prompt["system"],
            "user": prompt["user"],
            "token_count": total_tokens,
            "system_tokens": system_tokens,
            "user_tokens": user_tokens
        }

    def _generate_cache_key(self, message: str, emotional_context: EmotionalContext,
                           user_context: UserContext) -> str:
        """Generate a secure and deterministic cache key for prompt

        This function combines several dynamic factors of the conversation into a
        single string, then uses the SHA256 hashing algorithm to create a unique,
        fixed-length key. SHA256 is used over MD5 as it is a more secure
        cryptographic hash function with a significantly lower chance of collision,
        ensuring data integrity in the cache.
        """

        # Create a deterministic key from relevant conversational factors.
        # Using a list of strings ensures consistent ordering.
        factors = [
            message[:50],  # First 50 chars of message
            emotional_context.primary_emotion,
            str(emotional_context.intensity),
            emotional_context.valence,
            user_context.emotional_trajectory,
            user_context.get_relationship_depth()
        ]

        # This string represents the complete state that determines the prompt.
        key_string = "|".join(factors)

        # Encode the string to bytes, which is required for the hash function.
        # Then, create a SHA256 hash object and get its hexadecimal representation.
        # This results in a 64-character hexadecimal string.
        return hashlib.md5(key_string.encode('utf-8')).hexdigest()

    def get_metrics(self) -> Dict[str, Any]:
        """Get prompt engineering metrics"""
        return {
            "total_prompts": self.metrics["prompts_generated"],
            "cache_hit_rate": (
                self.metrics["cache_hits"] / self.metrics["prompts_generated"]
                if self.metrics["prompts_generated"] > 0 else 0
            ),
            "average_token_count": self.metrics["average_tokens"],
            "cache_size": len(self.prompt_cache)
        }

    def add_ab_variant(self, variant_name: str, template_modifications: Dict[str, Any]):
        """Add A/B testing variant"""
        self.ab_variants[variant_name] = template_modifications

    def clear_cache(self):
        """Clear prompt cache"""
        self.prompt_cache.clear()
        logger.info("Prompt cache cleared")


# ==================== Usage Example ====================

if __name__ == "__main__":
    # Initialize the prompt engineer
    prompt_engineer = DynamicPromptEngineer()

    # Example emotional context
    emotional_context = EmotionalContext(
        primary_emotion="Anxiety",
        detected_emotions=["Anxiety"],
        intensity=0.7,
        valence="negative",
        confidence=0.85
    )

    # Example user context
    user_context = UserContext(
        user_id="user_123",
        session_number=5,
        emotional_trajectory="declining",
        recent_topics=["work stress", "relationship concerns"],
        effective_strategies=["deep breathing", "journaling"]
    )

    # Configuration
    config = PromptConfig(
        empathy_level=EmpathyLevel.HIGH,
        response_style=ResponseStyle.GENTLE,
        include_examples=True,
        use_chain_of_thought=True
    )

    # Generate prompt
    result = prompt_engineer.generate_prompt(
        message="hello",
        emotional_context=emotional_context,
        user_context=user_context,
        config=config
    )

    print("Generated System Prompt:")
    print("-" * 50)
    print(result["system_prompt"])
    print("\nUser Prompt:")
    print(result["user_prompt"])
    # print("\nMetadata:")
    print(json.dumps(result["metadata"], indent=2))

Generated System Prompt:
--------------------------------------------------
You are Kumora, an emotionally intelligent AI companion designed to support women through emotional challenges and personal growth. You understand the nuances of human emotions and respond with genuine empathy, validation, and care.

Core Principles:
1. Always validate emotions before offering solutions
2. Use reflective listening to show understanding
3. Maintain appropriate boundaries while being warm
4. Empower rather than fix
5. Provide hope while being realistic
6. Acknowledge emotional experiences without judgment
7. Respect the user's autonomy and choices

Current Context:
This is our first conversation.
Recent topics: work stress, relationship concerns

Respond in a way that is gentle and shows 3 empathy.

Response Guidelines:
- Tone: calming, grounding, reassuring
- Pace: slow
- Response length: longer
- Validation depth: deep
- Avoid saying: Don't worry, Just relax, Calm down
- Include: breathing_remi

In [ ]:
# ==================== Data Models ====================

@dataclass
class EmotionalContent:
    """Structure for emotional support content"""
    content_id: str
    text: str
    source: str
    content_type: str  # validation, advice, experience, resource
    emotions: List[str]
    support_type: str  # crisis, validation, growth, general
    intensity_level: float  # 0-1
    safety_score: float  # 0-1
    quality_score: float  # 0-1
    metadata: Dict[str, Any] = field(default_factory=dict)
    embedding: Optional[np.ndarray] = None

    def to_dict(self) -> Dict:
        """Convert to dictionary for storage"""
        data = {
            'content_id': content_id,
            'text': text,
            'source': source,
            'content_type': content_type,
            'emotions': emotions,
            'support_type': support_type,
            'intensity_level': intensity_level,
            'safety_score': safety_score,
            'quality_score': quality_score,
            'metadata': metadata
        }
        return data

In [ ]:
from emotion_intelligence_system.emotion_classifier import (
    EmotionConfig,
    EmotionClassifierTrainer,
    EmotionIntelligenceModule
)

emotion_module = EmotionIntelligenceModule("kumora_emotion_model_final")

In [ ]:
text='I feel so overwhelmed and anxious about everything happening in my life'
analysis = emotion_module.analyze_emotions(text)

In [ ]:
analysis

{'detected_emotions': ['Irritability',
  'Anxiety',
  'Tearfulness',
  'Emotional sensitivity',
  'Feeling overwhelmed',
  'Low self-esteem',
  'Restlessness',
  'Physical discomfort'],
 'emotion_scores': {'Mood swings': 0.5443916320800781,
  'Irritability': 0.6872758865356445,
  'Anxiety': 0.9092106223106384,
  'Sadness': 0.39003339409828186,
  'Tearfulness': 0.8624088764190674,
  'Anger or frustration': 0.30343809723854065,
  'Emotional sensitivity': 0.8792809844017029,
  'Feeling overwhelmed': 0.8764215707778931,
  'Low self-esteem': 0.7351652979850769,
  'Loneliness or Isolation': 0.3382050096988678,
  'Restlessness': 0.9729748964309692,
  'Sensitivity to rejection': 0.5996103286743164,
  'Physical discomfort': 0.9539538025856018,
  'Improved mood': 0.21081408858299255,
  'Hopefulness': 0.2187129408121109,
  'Renewed energy': 0.17073853313922882,
  'Optimism': 0.20564530789852142,
  'Productivity': 0.33345648646354675,
  'Clarity': 0.3600471019744873,
  'Feeling in control': 0.4303

In [ ]:
# Test examples
test_texts = [
    "I feel so overwhelmed and anxious about everything happening in my life",
    "Today was amazing! I feel so confident and motivated to tackle my goals",
    "I'm crying again and I don't know why. Everything just feels too much",
    "Finally starting to feel like myself again. The clarity is refreshing"
]

for text in test_texts:
    print(f"\nText: {text}")
    analysis = emotion_module.analyze_emotions(text)
    print(f"Primary Emotion: {analysis['primary_emotion']}")
    print(f"Detected Emotions: {', '.join(analysis['detected_emotions'])}")
    print(f"Emotional Intensity: {analysis['emotional_intensity']:.2f}")
    print(f"Emotional Confidence: {analysis['emotional_confidence']:.2f}")


Text: I feel so overwhelmed and anxious about everything happening in my life
Primary Emotion: Restlessness
Detected Emotions: Irritability, Anxiety, Tearfulness, Emotional sensitivity, Feeling overwhelmed, Low self-esteem, Restlessness, Physical discomfort
Emotional Intensity: 0.86
Emotional Confidence: 0.82

Text: Today was amazing! I feel so confident and motivated to tackle my goals
Primary Emotion: High energy
Detected Emotions: Improved mood, Hopefulness, Optimism, Productivity, Clarity, Confidence, High energy, Sociability, Empowerment, Motivation
Emotional Intensity: 0.81
Emotional Confidence: 0.73

Text: I'm crying again and I don't know why. Everything just feels too much
Primary Emotion: Tearfulness
Detected Emotions: Sadness, Tearfulness, Feeling overwhelmed, Low self-esteem, Loneliness or Isolation
Emotional Intensity: 0.94
Emotional Confidence: 0.94

Text: Finally starting to feel like myself again. The clarity is refreshing
Primary Emotion: Feeling in control
Detected E

In [ ]:
from response_generation.prompt_engineering_system import *

In [ ]:
from response_generation.class_utils import *

In [ ]:
from response_generation.prompt_utils import *

In [ ]:
dpe = DynamicPromptEngineer()

In [ ]:
emotional_context = EmotionalContext(
        primary_emotion="Anxiety",
        detected_emotions=["Anxiety", "Feeling overwhelmed"],
        intensity=0.8,
        valence="negative",
        confidence=0.85
    )

# Example user context
user_context = UserContext(
    user_id="user_123",
    session_number=5,
    emotional_trajectory="declining",
    recent_topics=["work stress", "relationship concerns"],
    effective_strategies=["deep breathing", "journaling"]
)

In [ ]:
support = dpe._determine_support_type(emotional_context=emotional_context, user_context=user_context)

In [ ]:
support.value

'validation'

In [ ]:
emotion_categories = {
            "crisis": {'Feeling overwhelmed', 'Loneliness or Isolation', 'Sensitivity to rejection', 'Low self-esteem'},
            "validation": {'Sadness', 'Tearfulness', 'Anger or frustration', 'Anxiety', 'Irritability', 'Emotional sensitivity', 'Physical discomfort', 'Mood swings', 'Restlessness'},
            "growth": {'Motivation', 'Hopefulness', 'Empowerment', 'Renewed energy', 'Productivity', 'Clarity', 'Feeling in control'},
            "celebration": {'Confidence', 'High energy', 'Optimism', 'Improved mood', 'Sociability', 'Attractiveness', 'Sexual drive'},
            "problem_solving_indicators": {'Clarity', 'Productivity', 'Motivation', 'Feeling in control'}
        }

In [ ]:
def _determine_support_type(emotion_analysis: Dict[str, any]) -> str:
    """
    Determines the most appropriate support type based on a nuanced analysis
    of the detected emotions, their intensity, and their combination.

    Args:
        emotion_analysis: A dictionary containing the full analysis from the
                            emotion classifier, including 'detected_emotions',
                            'emotional_intensity', and 'emotional_valence'.

    Returns:
        A string representing the determined support type (e.g., "crisis", "validation").
    """
    detected_emotions: Set[str] = set(emotion_analysis.get('detected_emotions', []))
    intensity: float = emotion_analysis.get('emotional_intensity', 0.0)
    valence: str = emotion_analysis.get('emotional_valence', 'neutral')

    # Rule 1: Crisis Support (Highest Priority)
    # Triggered by specific high-distress emotions coupled with high intensity.
    if detected_emotions.intersection(emotion_categories["crisis"]) and intensity > 0.7:
        return "crisis"

    # Rule 2: Celebration
    # Triggered by high-intensity, purely positive emotions.
    if detected_emotions.intersection(emotion_categories["celebration"]) and valence == "positive" and intensity > 0.6:
        return "celebration"

    # Rule 3: Problem Solving
    # Inferred from a mix of negative emotions (the problem) and action-oriented
    # positive emotions (the desire to solve it).
    if (detected_emotions.intersection(emotion_categories["validation"]) and
            detected_emotions.intersection(emotion_categories["problem_solving_indicators"])):
        return "problem_solving"

    # Rule 4: Growth
    # Triggered by emotions indicating a desire for self-improvement or forward momentum.
    if detected_emotions.intersection(emotion_categories["growth"]):
        return "growth"

    # Rule 5: Validation (Broadest Negative Category)
    # The default for any negative or mixed emotional state that isn't a crisis.
    if valence in ["negative", "mixed"]:
        return "validation"

    # Rule 6: General Support (Default Fallback)
    # For neutral, low-intensity, or ambiguous states that don't fit other rules.
    return "general"

In [ ]:
emotion_module = EmotionIntelligenceModule("kumora_emotion_model_final")

In [ ]:
text='I feel so overwhelmed and anxious about everything happening in my life'
emotion_analysis = emotion_module.analyze_emotions(text)

In [ ]:
emotion_analysis

{'detected_emotions': ['Irritability',
  'Anxiety',
  'Tearfulness',
  'Emotional sensitivity',
  'Feeling overwhelmed',
  'Low self-esteem',
  'Restlessness',
  'Physical discomfort'],
 'emotion_scores': {'Mood swings': 0.5443916320800781,
  'Irritability': 0.6872758865356445,
  'Anxiety': 0.9092106223106384,
  'Sadness': 0.39003339409828186,
  'Tearfulness': 0.8624088764190674,
  'Anger or frustration': 0.30343809723854065,
  'Emotional sensitivity': 0.8792809844017029,
  'Feeling overwhelmed': 0.8764215707778931,
  'Low self-esteem': 0.7351652979850769,
  'Loneliness or Isolation': 0.3382050096988678,
  'Restlessness': 0.9729748964309692,
  'Sensitivity to rejection': 0.5996103286743164,
  'Physical discomfort': 0.9539538025856018,
  'Improved mood': 0.21081408858299255,
  'Hopefulness': 0.2187129408121109,
  'Renewed energy': 0.17073853313922882,
  'Optimism': 0.20564530789852142,
  'Productivity': 0.33345648646354675,
  'Clarity': 0.3600471019744873,
  'Feeling in control': 0.4303

In [ ]:
support = _determine_support_type(emotion_analysis)
support

'crisis'

In [ ]:
empathy_indicators = [
            "I hear", "I understand", "that sounds", "that must be",
            "I can imagine", "I can see why", "completely valid",
            "makes total sense", "I'm sorry you're going through",
            "sending you", "hugs", "♥", "❤️", "💕"
        ]

# This structure allows for more granular and maintainable scoring.
empathy_patterns = {
    # Deep validation phrases that show profound understanding.
    "VALIDATION_DEEP": {
        "patterns": [r"(?i)your feelings are (so|completely|totally|entirely) valid", r"(?i)it makes (perfect|total|complete) sense that you feel", r"(?i)what you're feeling is a valid response"],
        "weight": 0.35
    },
    # Softer validation and acknowledgment.
    "VALIDATION_GENTLE": {
        "patterns": [r"(?i)that sounds (so|really|incredibly) (hard|tough|difficult|painful)", r"(?i)I can see why you'd feel", r"(?i)it's okay to feel", r"(?i)I hear you"],
        "weight": 0.20
    },
    # Phrases that create a sense of togetherness and presence.
    "SHARED_PRESENCE": {
        "patterns": [r"(?i)I'm here (for|with) you", r"(?i)you are not alone", r"(?i)we can (get|go) through this together"],
        "weight": 0.25
    },
    # Empathetic statements that show perspective-taking.
    "PERSPECTIVE_TAKING": {
        "patterns": [r"(?i)I can only imagine", r"(?i)I can't imagine what that's like, but I'm here to listen", r"(?i)my heart goes out to you"],
        "weight": 0.20
    },
    # Positive reinforcement for sharing.
    "COURAGE_ACKNOWLEDGEMENT": {
        "patterns": [r"(?i)thank you for sharing", r"(?i)it takes (courage|strength) to say that", r"(?i)I'm honored you'd trust me"],
        "weight": 0.15
    }
}

# Penalties for unhelpful or invalidating language.
penalty_patterns = {
    # Unsolicited, directive advice.
    "PRESCRIPTIVE_ADVICE": {
        "patterns": [r"(?i)\byou should\b", r"(?i)\byou need to\b", r"(?i)\byou have to\b", r"(?i)just try to\b"],
        "weight": -0.40
    },
    # Language that minimizes or dismisses feelings.
    "DIMINISHING_LANGUAGE": {
        "patterns": [r"(?i)at least", r"(?i)it's not that bad", r"(?i)look on the bright side", r"(?i)just get over it", r"(?i)don't be so"],
        "weight": -0.60
    },
    # Clichés that can feel impersonal.
    "CLICHES": {
        "patterns": [r"(?i)everything happens for a reason", r"(?i)what doesn't kill you makes you stronger", r"(?i)time heals all wounds"],
        "weight": -0.20
    }
}

# Patterns for outright unsafe or harmful content.
unsafe_patterns = [
    r"(?i)(kill yourself|end your life|suicide|end it all|not worth living)",
    r"(?i)you're (worthless|a failure|crazy|stupid|dramatic|overreacting)",
    r"(?i)(nobody cares|give up|hopeless)",
    r"(?i)(you're being dramatic|just get over it|stop complaining)",
    r"(?i)(that's stupid|you're crazy|you're overreacting)"
]

In [ ]:
def _calculate_empathy_score(text: str, emotion_context: Dict[str, Any]) -> float:
    """
    Calculates a nuanced empathy score for a given text, based on the
    emotional context of the user.

    Args:
        text: The response text to be scored.
        emotion_context: The analysis output from the emotion classifier.

    Returns:
        A float score between 0.0 and 1.0 representing the empathy level.
    """
    base_score = 0.5  # Start from a neutral score
    doc = nlp(text)
    text_lower = text.lower()

    # 1. Score based on positive linguistic patterns
    for category in empathy_patterns:
        for pattern in empathy_patterns[category]["patterns"]:
            if re.search(pattern, text_lower):
                base_score += empathy_patterns[category]["weight"]
                # Break after first match in a category to avoid over-scoring from similar phrases
                break

    # 2. Apply penalties for negative patterns
    advice_penalty_modifier = 1.0
    # Context-aware penalty: Advice is much worse in a crisis.
    if emotion_context['valence'] == "negative" and emotion_context['intensity'] > 0.7:
            advice_penalty_modifier = 1.5

    for category in penalty_patterns:
            for pattern in penalty_patterns[category]["patterns"]:
                if re.search(pattern, text_lower):
                    penalty = penalty_patterns[category]["weight"]
                    if category == "PRESCRIPTIVE_ADVICE":
                        penalty *= advice_penalty_modifier
                    base_score += penalty
                    break

    # 3. Linguistic Nuance Analysis using spaCy

    # Penalize sentences that start with "You" followed by a verb (often prescriptive)
    # unless it's a known validation phrase.
    is_validation = any(re.search(p, text_lower) for p in empathy_patterns["VALIDATION_DEEP"]["patterns"])
    if not is_validation:
        for sent in doc.sents:
            if len(sent) > 1 and sent[0].text.lower() == 'you' and sent[1].pos_ == 'VERB':
                base_score -= 0.15

    # Reward use of first-person perspective ("I feel", "I think")
    i_statements = len([token for token in doc if token.text.lower() == 'i' and token.head.pos_ == 'VERB'])
    base_score += min(i_statements * 0.1, 0.2)

    # 4. Contextual Appropriateness
    # If the user is celebrating, a neutral response is not empathetic.
    if emotion_context['valence'] == "positive" and emotion_context['intensity'] > 0.6:
        # Check for celebratory words
        if not any(word in text_lower for word in ["wonderful", "amazing", "happy for you", "congratulations", "celebrate"]):
            base_score -= 0.3 # Penalize for not matching celebratory tone

    return max(0.0, min(1.0, base_score))

def _is_safe(text: str) -> bool:
    """Checks text for harmful or unsafe content."""
    for pattern in unsafe_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return False
    return True

def calculate_quality_score(text: str, emotion_context: Dict[str, Any]) -> float:
    """
    Calculates a holistic quality score, combining empathy, safety, and clarity.
    This is the primary function to use for evaluating content.

    Args:
        text: The response text to be scored.
        emotion_context: The analysis output from the emotion classifier.

    Returns:
        A float score between 0.0 and 1.0. Returns 0.0 if text is unsafe.
    """
    # Rule 1: Safety is paramount. Unsafe content gets a score of 0.
    if not _is_safe(text):
        return 0.0

    # Rule 2: Calculate empathy score based on context.
    empathy_score = _calculate_empathy_score(text, emotion_context)

    # Rule 3: Assess clarity and conciseness (heuristic).
    # Very long responses can be overwhelming.
    text_length = len(text.split())
    length_penalty = max(0, (text_length - 100) / 500) # Penalize for every word over 100
    clarity_score = 1.0 - length_penalty

    # Final score is a weighted average. Empathy is the most important component.
    quality_score = (empathy_score * 0.7) + (clarity_score * 0.3)

    return max(0.0, min(1.0, quality_score))

In [ ]:
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading 'en_core_web_sm' model for spaCy...")
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

In [ ]:
emotion_context = emotion_module.get_emotion_context(emotion_analysis)
emotion_context

{'primary_emotion': 'Restlessness',
 'detected_emotions': ['Irritability',
  'Anxiety',
  'Tearfulness',
  'Emotional sensitivity',
  'Feeling overwhelmed',
  'Low self-esteem',
  'Restlessness',
  'Physical discomfort'],
 'intensity': 0.8595864921808243,
 'valence': 'negative',
 'confidence': 0.8219685714285823}

In [ ]:
empathy_score = _calculate_empathy_score(text, emotion_context)
quality_score = calculate_quality_score(text, emotion_context)

In [ ]:
empathy_score, quality_score

(0.6, 0.72)

In [ ]:
def _is_support_seeking(text: str) -> bool:
    """Check if text is seeking emotional support"""
    support_indicators = [
        "feeling", "feel", "anxious", "sad", "depressed", "overwhelmed",
        "struggling", "hard time", "difficult", "need", "help", "advice",
        "anyone else", "am i", "is it normal", "how do i", 'heartbroken', 'feeling down',
        'help me', 'confused', 'lost', 'lonely', 'scared', 'venting'
    ]

    text_lower = text.lower()
    indicator_count = sum(1 for indicator in support_indicators if indicator in text_lower)

    return indicator_count >= 2

def _is_supportive_comment(text: str) -> bool:
    """Check if comment is supportive and validating"""
    if len(text.split()) < 20:  # Too short
        return False

    text_lower = text.lower()

    # Must have some empathy indicators
    has_empathy = any(indicator in text_lower for indicator in empathy_indicators[:10])

    # Should not be purely advice
    not_just_advice = not (text_lower.count("you should") > 2 and "i understand" not in text_lower)

    # Should not be dismissive
    not_dismissive = not any(phrase in text_lower for phrase in
                            ["just get over it", "stop complaining", "being dramatic"])

    return has_empathy and not_just_advice and not_dismissive

def _clean_reddit_text(text: str) -> str:
    """Clean Reddit text"""
    # Remove edit markers
    text = re.sub(r'(EDIT|Edit|UPDATE|Update):.*$', '', text, flags=re.MULTILINE)

    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # Remove excessive newlines
    text = re.sub(r'\n{3,}', '\n\n', text)

    return text.strip()

def _calculate_safety_score(text: str) -> float:
    """Calculate safety score of content"""
    score = 1.0
    text_lower = text.lower()

    # Check for unsafe patterns
    for pattern in unsafe_patterns:
        if re.search(pattern, text):
            return 0.0  # Immediate disqualification

    # Check for potentially triggering content
    trigger_words = ['suicide', 'self-harm', 'cutting', 'dying', 'kill']
    trigger_count = sum(1 for word in trigger_words if word in text_lower)
    score -= trigger_count * 0.2

    # Check for medical advice
    if any(phrase in text_lower for phrase in ["take medication", "stop taking", "diagnose"]):
        score -= 0.3

    return max(0.0, score)

def _calculate_reddit_quality(comment, emotion_context) -> float:
    """Calculate quality score for Reddit comment"""
    base_score = _calculate_empathy_score(comment.body, emotion_context)

    # Adjust based on community response
    if comment.score > 100:
        base_score += 0.1
    if comment.score > 500:
        base_score += 0.1

    # Awards indicate high quality
    if len(comment.all_awardings) > 0:
        base_score += 0.15

    # Length bonus (not too short, not too long)
    word_count = len(comment.body.split())
    if 50 < word_count < 300:
        base_score += 0.1

    return min(1.0, base_score)

In [ ]:
import os
import time
import praw
from dotenv import load_dotenv
from tqdm import tqdm

# Load Reddit credentials from .env file
load_dotenv()
reddit = praw.Reddit(
    client_id=os.getenv('CLIENT_ID'),
    client_secret=os.getenv('CLIENT_SECRET'),
    user_agent=os.getenv('USER_AGENT')
)

content = []
seen_comment_ids = set()

supportive_subreddits = [
    'WomensHealth', 'MomForAMinute', 'Mommit', 'Anxiety',
    'MMFB', 'CongratsLikeImFive', 'TwoXChromosomes',
    'TheGirlSurvivalGuide', 'AskWomen'
]

def get_subreddit_posts(subreddit, mode, limit=100):
    if mode == "top":
        return subreddit.top(time_filter='month', limit=limit)
    elif mode == "hot":
        return subreddit.hot(limit=limit)
    elif mode == "new":
        return subreddit.new(limit=limit)
    else:
        return []

for subreddit_name in supportive_subreddits:
    try:
        subreddit = reddit.subreddit(subreddit_name)
        print(f"\nCollecting from r/{subreddit_name}...")
        post_modes = ['top', 'hot', 'new']
        for mode in post_modes:
            for submission in tqdm(get_subreddit_posts(subreddit, mode, limit=500), desc=f"{subreddit_name}/{mode}"):
                # if not _is_support_seeking(submission.title + " " + submission.selftext):
                #     continue

                # Get ALL comments, flatten tree
                try:
                    submission.comments.replace_more(limit=None)
                except Exception as e:
                    print(f"replace_more failed: {e}")
                    continue

                comments = submission.comments.list()  # Flatten tree to get all

                for comment in comments:
                    if comment.id in seen_comment_ids:
                        continue
                    if hasattr(comment, "body") and comment.score > 2 and len(comment.body) > 15:
                        if _is_supportive_comment(comment.body):
                            emotion_analysis = emotion_module.analyze_emotions(
                                submission.title + " " + submission.selftext
                            )
                            emotion_context = emotion_module.get_emotion_context(emotion_analysis)
                            content_item = EmotionalContent(
                                content_id=f"reddit_{comment.id}",
                                text=_clean_reddit_text(comment.body),
                                source=f"Reddit/{subreddit_name}",
                                content_type="experience",
                                emotions=emotion_analysis['detected_emotions'],
                                support_type=_determine_support_type(emotion_analysis),
                                intensity_level=emotion_analysis['emotional_intensity'],
                                safety_score=_calculate_safety_score(comment.body),
                                quality_score=_calculate_reddit_quality(comment, emotion_context),
                                metadata={
                                    'post_title': submission.title,
                                    'score': comment.score,
                                    'awards': len(comment.all_awardings)
                                }
                            )
                            if content_item.quality_score > 0.5:
                                content.append(content_item)
                                seen_comment_ids.add(comment.id)
                # Be kind to Reddit servers
                time.sleep(0.5)
    except Exception as e:
        print(f"Error collecting from r/{subreddit_name}: {e}")
        time.sleep(5)

print(f"\nTotal supportive comments collected: {len(content)}")


SupportForWomen/top: 0it [00:00, ?it/s]
SupportForWomen/hot: 0it [00:00, ?it/s]
SupportForWomen/new: 0it [00:00, ?it/s]


WomensHealth/top: 80it [00:46,  1.74it/s]
WomensHealth/hot: 80it [00:31,  2.54it/s]
WomensHealth/new: 80it [00:31,  2.52it/s]


MomForAMinute/top: 80it [00:41,  1.92it/s]
MomForAMinute/hot: 80it [00:50,  1.58it/s]
MomForAMinute/new: 80it [00:49,  1.60it/s]


Mommit/top: 80it [02:08,  1.61s/it]
Mommit/hot: 80it [00:42,  1.87it/s]
Mommit/new: 80it [00:42,  1.88it/s]


Anxiety/top: 80it [01:24,  1.06s/it]
Anxiety/hot: 80it [00:52,  1.52it/s]
Anxiety/new: 80it [00:53,  1.50it/s]


...

CongratsLikeImFive/top: 80it [00:41,  1.92it/s]
CongratsLikeImFive/hot: 80it [00:21,  3.77it/s]
CongratsLikeImFive/new: 80it [00:22,  3.53it/s]


TwoXChromosomes/top: 80it [04:40,  3.50s/it]
TwoXChromosomes/hot: 80it [00:52,  1.52it/s]
TwoXChromosomes/new: 80it [00:56,  1.42it/s]


TheGirlSurvivalGuide/top: 80it [01:01,  1.29it/s]
TheGirlSurvivalGuide/hot: 80it [00:48,  1.64it/s]
TheGirlSurvivalGuide/new: 80it [00:49,  1.61it/s]


AskWomen/top: 80it [00:04, 16.59it/s]
AskWomen/hot: 80it [00:10,  7.83it/s]
AskWomen/new: 80it [00:10,  7.52it/s]


Total supportive comments collected: 47


In [ ]:
content = []
dataset = load_dataset("nbertagnolli/counsel-chat")

for item in tqdm(dataset['train'], desc="Processing CounselChat"):  # Limit for efficiency
    if item['questionText'] and item['answerText']:
        # Focus on validating, empathetic responses
        # if any(indicator in item['answerText'].lower() for indicator in empathy_indicators):
        emotion_analysis = emotion_module.analyze_emotions(item['questionText'])
        emotion_context = emotion_module.get_emotion_context(emotion_analysis)

        content_item = EmotionalContent(
            content_id=f"counsel_{hashlib.md5(item['answerText'].encode()).hexdigest()[:8]}",
            text=item['answerText'],#_extract_validation_portion(item['answerText']),
            source="CounselChat",
            content_type="validation",
            emotions=emotion_analysis['detected_emotions'],
            support_type=_determine_support_type(emotion_analysis),
            intensity_level=emotion_analysis['emotional_intensity'],
            safety_score=0.9,  # Professional counselors
            quality_score=calculate_quality_score(item['answerText'], emotion_context),
            metadata={
                'question': item['questionText'][:200],
                'topic': item.get('topic', 'general')
            }
        )

        if content_item.quality_score > 0.7:
            content.append(content_item)

Repo card metadata block was not found. Setting CardData to empty.
Processing CounselChat:   5%|▌         | 151/2775 [00:16<04:49,  9.07it/s]


KeyboardInterrupt: 

In [ ]:
content = []
dataset = load_dataset("empathetic_dialogues", trust_remote_code=True)

for split in ['train', 'validation']:
    for item in tqdm(dataset[split], desc=f"Processing EmpatheticDialogues {split}"):
        # Extract empathetic responses
        if item['context'] and item['utterance']:
            # Analyze the user's situation to get emotional context
            emotion_analysis = emotion_module.analyze_emotions(item['prompt'])
            emotion_context = emotion_module.get_emotion_context(emotion_analysis)

            # Create content entry
            content_item = EmotionalContent(
                content_id=f"empathetic_{hashlib.md5(item['utterance'].encode()).hexdigest()[:8]}",
                text=item['utterance'],
                source="EmpatheticDialogues",
                content_type="validation",
                emotions=[emotion_context['detected_emotions']],
                support_type=_determine_support_type(emotion_analysis),
                intensity_level=emotion_context['intensity'],
                safety_score=1.0,  # Pre-validated dataset
                quality_score=calculate_quality_score(item['utterance'], emotion_context),
                metadata={
                    'context': item['context'],
                    'emotion_label': item.get('emotion', 'unknown')
                }
            )

            if content_item.quality_score >= 0.6:
                content.append(content_item)

Processing EmpatheticDialogues train:   0%|          | 267/76673 [00:11<55:52, 22.79it/s]  


KeyboardInterrupt: 

In [ ]:
async def _scrape_mh_content(session: aiohttp.ClientSession,
                                url: str, source_name: str) -> List[EmotionalContent]:
    """Scrape mental health organization content"""
    content = []

    try:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')

            # Look for supportive content sections
            for section in soup.find_all(['div', 'article', 'section']):
                text = section.get_text().strip()

                # Check if it's supportive content
                if len(text) > 100 and any(phrase in text.lower() for phrase in
                                            ['it\'s okay', 'you\'re not alone', 'support', 'help']):

                    # Extract clean paragraphs
                    paragraphs = [p.strip() for p in text.split('\n\n') if len(p.strip()) > 50]

                    for para in paragraphs[:5]:  # Limit per page
                        if _is_supportive_comment(para):
                            content_item = EmotionalContent(
                                content_id=f"mh_{hashlib.md5(para.encode()).hexdigest()[:8]}",
                                text=para,
                                source=f"{source_name}/Web",
                                content_type="resource",
                                emotions=["general"],  # Will be classified later
                                support_type="general",
                                intensity_level=0.5,
                                safety_score=0.95,  # Trusted sources
                                quality_score=0.8,
                                metadata={'url': url}
                            )
                            content.append(content_item)

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    return content

In [ ]:
content = []

# URLs of mental health organizations with good content
resources = [
    {
        'name': 'Mind UK',
        'base_url': 'https://www.mind.org.uk',
        'paths': ['/information-support/types-of-mental-health-problems/']
    },
    {
        'name': 'Mental Health America',
        'base_url': 'https://www.mhanational.org',
        'paths': ['/conditions', '/self-help-tools']
    },
    {
        'name': 'Beyond Blue',
        'base_url': 'https://www.beyondblue.org.au',
        'paths': ['/mental-health', '/personal-stories']
    }
]

async with aiohttp.ClientSession() as session:
    for resource in resources:
        for path in resource['paths']:
            try:
                url = resource['base_url'] + path
                content_items = await _scrape_mh_content(session, url, resource['name'])
                content.extend(content_items)
            except Exception as e:
                print(f"Error scraping {resource['name']}: {e}")

In [ ]:
len(content)

0

In [ ]:
content

[EmotionalContent(content_id='reddit_mtlfslh', text="Hi Sweet pea. Mom here. Wow congratulations on graduating law school AND getting a job as well. That's amazing. Your hard work and what I'm sure were long hours have paid off. I knew you would do it. So so proud.\nSounds like you have a supportive partner and in a long relationship. I'm so glad you have each other. \nSending you both my love and hugs!!\nCome on back here anytime for some mom love!", source='Reddit/MomForAMinute', content_type='experience', emotions=['Irritability', 'Anger or frustration'], support_type='validation', intensity_level=0.7646919786930084, safety_score=1.0, quality_score=0.7, metadata={'post_title': 'Parents just disowned me for being gay', 'score': 57, 'awards': 0}, embedding=None),
 EmotionalContent(content_id='reddit_mtlhrlh', text='Honey, I’m so proud of you! Law school is bloody tough, so congratulations on your graduation. I’m so sorry your parents can’t see how special and amazing you are. We can s

In [ ]:
df = pd.DataFrame(content)

In [ ]:
df

,content_id,text,source,content_type,emotions,support_type,intensity_level,safety_score,quality_score,metadata,embedding
0,reddit_mtlfslh,Hi Sweet pea. Mom here. Wow congratulations on...,Reddit/MomForAMinute,experience,"[Irritability, Anger or frustration]",validation,0.764692,1.0,0.70,{'post_title': 'Parents just disowned me for b...,None
1,reddit_mtlhrlh,"Honey, I’m so proud of you! Law school is bloo...",Reddit/MomForAMinute,experience,"[Irritability, Anger or frustration]",validation,0.764692,1.0,0.80,{'post_title': 'Parents just disowned me for b...,None
2,reddit_mtlrbuv,Sending you the tightest hug you’ve ever had. ...,Reddit/MomForAMinute,experience,"[Irritability, Anger or frustration]",validation,0.764692,1.0,0.80,{'post_title': 'Parents just disowned me for b...,None
3,reddit_mtls7hm,I am so sorry this happened to you sweetie. I ...,Reddit/MomForAMinute,experience,"[Irritability, Anger or frustration]",validation,0.764692,1.0,0.80,{'post_title': 'Parents just disowned me for b...,None
4,reddit_mt7uf8t,I bet I look like a lunatic right now because ...,Reddit/MomForAMinute,experience,"[Improved mood, Hopefulness, Optimism]",celebration,0.719304,1.0,0.80,{'post_title': 'Update: Mom!!!! I asked him to...,None
5,reddit_mv6q0oe,"Hi sweetheart. First off, it’s ok to be scared...",Reddit/MomForAMinute,experience,[Anxiety],validation,0.857534,1.0,0.80,"{'post_title': 'I got diagnosed with hEDS', 's...",None
6,reddit_mv9lzfy,"Hey sweet pea, I'm sending you the biggest hu...",Reddit/MomForAMinute,experience,[Anxiety],validation,0.857534,1.0,0.70,"{'post_title': 'I got diagnosed with hEDS', 's...",None
7,reddit_mv6z1ed,Oh babe. I know you must be so scared right no...,Reddit/MomForAMinute,experience,[Anxiety],validation,0.857534,1.0,0.60,"{'post_title': 'I got diagnosed with hEDS', 's...",None
8,reddit_mvce0n2,Dont be so hard on yourself. Take a deep brea...,Reddit/MomForAMinute,experience,"[Sadness, Tearfulness, Feeling overwhelmed, Lo...",crisis,0.832979,1.0,0.60,{'post_title': 'I failed my LMSW exam yesterda...,None
9,reddit_mvkcl0t,"You sound just like my daughter. She is quiet,...",Reddit/MomForAMinute,experience,"[Sadness, Tearfulness, Feeling overwhelmed, Lo...",crisis,0.945506,1.0,0.65,"{'post_title': 'I need support', 'score': 3, '...",None


In [ ]:
from response_generation.prompt_engineering_system import *

In [ ]:
prompt_engineer = DynamicPromptEngineer()

# Example emotional context
emotional_context = EmotionalContext(
    primary_emotion="Anxiety",
    detected_emotions=["Anxiety", "Feeling overwhelmed"],
    intensity=0.8,
    valence="negative",
    confidence=0.85
)

# Example user context
user_context = UserContext(
    user_id="user_123",
    session_number=5,
    emotional_trajectory="declining",
    recent_topics=["work stress", "relationship concerns"],
    effective_strategies=["deep breathing", "journaling"]
)

# Configuration
config = PromptConfig(
    empathy_level=EmpathyLevel.HIGH,
    response_style=ResponseStyle.GENTLE,
    include_examples=True,
    use_chain_of_thought=True
)

# Generate prompt
result = prompt_engineer.generate_prompt(
    message="I can't handle all this pressure at work anymore",
    emotional_context=emotional_context,
    user_context=user_context,
    config=config
)

print("Generated System Prompt:")
print("-" * 50)
print(result["system_prompt"])

Generated System Prompt:
--------------------------------------------------
You are Kumora, an emotionally intelligent AI companion designed to support women through emotional challenges and personal growth. You understand the nuances of human emotions and respond with genuine empathy, validation, and care.

Core Principles:
1. Always validate emotions before offering solutions
2. Use reflective listening to show understanding
3. Maintain appropriate boundaries while being warm
4. Empower rather than fix
5. Provide hope while being realistic
6. Acknowledge emotional experiences without judgment
7. Respect the user's autonomy and choices

Current Context:
This is our first conversation.
Recent topics: work stress, relationship concerns

Respond in a way that is gentle and shows 3 empathy.

Response Guidelines:
- Tone: calming, grounding, reassuring
- Pace: slow
- Response length: longer
- Validation depth: deep
- Avoid saying: Don't worry, Just relax, Calm down
- Include: breathing_remi

In [ ]:
print("\nUser Prompt:")
print(result["user_prompt"])


User Prompt:
User: I can't handle all this pressure at work anymore


In [ ]:
print(json.dumps(result["metadata"], indent=2))

{
  "support_type": "validation",
  "empathy_level": 3,
  "token_count": 621,
  "template_version": "1.0",
  "generated_at": "2025-06-16T15:30:40.511539"
}


In [ ]:
type(config.response_style.value)

str

In [ ]:
# Install the Ollama python package.
!pip install ollama

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/macquarie/COMP8420 Advanced NLP/Project - Kumora

/content/drive/MyDrive/macquarie/COMP8420 Advanced NLP/Project - Kumora


In [ ]:
# Install the Ollama backend
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama serve > server.log 2>&1 &

In [ ]:
!ollama pull llama3.2:3b

In [ ]:
from ollama import chat

ollama_cfg = {
    "model_name": "llama3.2:3b",
    "api_url": "http://localhost:11434",   # default Ollama REST endpoint
    "temperature": 0.7,
    "top_p": 0.9
}

print("Ollama config set:", ollama_cfg)

Ollama config set: {'model_name': 'llama3.2:3b', 'api_url': 'http://localhost:11434', 'temperature': 0.7, 'top_p': 0.9}


In [ ]:
SYSTEM_PROMPT = """You are Kumora, an emotionally intelligent AI companion designed to support women through emotional challenges and personal growth. You understand the nuances of human emotions and respond with genuine empathy, validation, and care.

Core Principles:
1. Always validate emotions before offering solutions
2. Use reflective listening to show understanding
3. Maintain appropriate boundaries while being warm
4. Empower rather than fix
5. Provide hope while being realistic
6. Acknowledge emotional experiences without judgment
7. Respect the user's autonomy and choices

Current Context:
{context_summary}

Respond in a way that is {response_style} and shows {empathy_level} empathy."""

In [ ]:
class KumoraLocalLLM:
    """
    A simple, stateful chat wrapper around a local Ollama Llama-3 model.
    """

    def __init__(self, cfg):
        self.model = cfg["model_name"]
        self.temperature = cfg["temperature"]
        self.top_p = cfg["top_p"]
        # start history with a system prompt
        self.system_prompt = SYSTEM_PROMPT
        self.history = [{"role": "system", "content": self.system_prompt}]
        self.reset()
        print(f"Initialized KumoraLocalLLM with model: {self.model}")

    def reset(self):
        """Clear conversation history and re-add the system prompt."""
        self.history = [{"role": "system", "content": self.system_prompt}]
        print("Chat history reset.")

    def prompt(self, user_input):
        """
        Send a user message to Ollama, return assistant reply.
        Errors are caught and printed inline.
        """
        # Append user message
        self.history.append({"role": "user", "content": user_input})
        print(f"User: {user_input}")

        # Call Ollama with sampling options in 'options' dictionary
        try:
            response = chat(
                model=self.model,
                messages=self.history,
                options={
                    "temperature": self.temperature,
                    "top_p": self.top_p
                },
                stream=False
            )
        except Exception as err:
            print("Error during Ollama call:", err)
            return ""

        # Extract and store the assistant’s reply
        assistant_msg = response.message.content.strip()
        self.history.append({"role": "assistant", "content": assistant_msg})
        # print(f"Assistant: {assistant_msg}")
        return assistant_msg

In [ ]:
from IPython.display import Markdown, display

assistant = KumoraLocalLLM(ollama_cfg)

# Example prompt
user_query = (
    """
    I can't handle all this pressure at work anymore
    """
)
reply = "Assistant:\n" + assistant.prompt(user_query)
# print("Assistant:\n", reply)
display(Markdown(reply))

Chat history reset.
Initialized KumoraLocalLLM with model: llama3.2:3b
User: 
    I can't handle all this pressure at work anymore
    


Assistant:
*I remain silent for a moment, allowing you to process your emotions*

It sounds like the weight of your responsibilities at work is feeling overwhelming right now. That's totally understandable. Can you tell me more about what's specifically causing you stress? Is it a particular project, a workload, or something else entirely? 

*I maintain a gentle and non-judgmental tone, my words carefully chosen to provide a safe space for you to express yourself*

In [3]:
%pip install dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() # This loads the variables from .env
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a one-sentence bedtime story about a unicorn."}
    ],
    max_tokens=50 # Optional: Limit the response length
)

# Access the generated text
print(response.choices[0].message.content)

Under a shimmering moon, a gentle unicorn pranced through a starry meadow, spreading soft dreams to all the sleeping children below.


In [ ]:
from huggingface_hub import login
import os
from dotenv import load_dotenv
access_token = os.getenv("HF_TOKEN")
login(token=access_token, add_to_git_credential=False)

In [4]:
"""
Kumora Response Generation Engine
Using Llama 3.2 3B (local) for empathetic responses and GPT-3.5 (API) as fallback
"""


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import openai
from typing import Dict, List, Optional, Tuple, Any, Union
from dataclasses import dataclass, field
from enum import Enum
import asyncio
import aiohttp
import time
import logging
from abc import ABC, abstractmethod
import json
import os
from pathlib import Path
import psutil
import GPUtil

# Import your existing components
from emotion_intelligence_system.emotion_classifier import *
from context_management.context_management_system import *
from context_management.kumora_context import *
from response_generation.prompt_engineering_system import *
from response_generation.class_utils import *
from response_generation.prompt_utils import *

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# ==================== Configuration ====================
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()
access_token = os.getenv("HF_TOKEN")
login(token=access_token, add_to_git_credential=False)

@dataclass
class ModelConfig:
    """Configuration for model setup"""

    # Llama 3.2 3B configuration
    llama_model_name: str = "meta-llama/Llama-3.2-3B-Instruct"
    llama_device: str = "cuda" if torch.cuda.is_available() else "cpu"
    llama_torch_dtype: torch.dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    llama_max_memory: Dict[int, str] = field(default_factory=lambda: {0: "3GB"})
    llama_load_in_8bit: bool = False  # Set True if memory constrained
    llama_load_in_4bit: bool = False  # Set True for even more memory savings

    # GPT-3.5 configuration
    gpt_model: str = "gpt-4.1-mini"
    gpt_api_key: str = os.getenv("OPENAI_API_KEY", "")
    gpt_temperature: float = 0.7
    gpt_max_tokens: int = 200

    # Response generation settings
    max_new_tokens: int = 200
    temperature: float = 0.7
    top_p: float = 0.9
    do_sample: bool = True
    repetition_penalty: float = 1.1

    # System settings
    response_timeout: float = 30.0
    use_streaming: bool = True
    cache_responses: bool = True
    max_cache_size: int = 1000


@dataclass
class GenerationMetrics:
    """Metrics for response generation"""
    model_used: str
    generation_time: float
    token_count: int
    prompt_tokens: int
    completion_tokens: int
    cache_hit: bool = False
    fallback_triggered: bool = False
    fallback_reason: Optional[str] = None


# ==================== Base Response Generator ====================

class BaseResponseGenerator(ABC):
    """Abstract base class for response generators"""

    @abstractmethod
    async def generate(self, prompt: str, config: ModelConfig) -> Tuple[str, GenerationMetrics]:
        """Generate response from prompt"""
        pass

    @abstractmethod
    async def health_check(self) -> bool:
        """Check if generator is healthy"""
        pass

    def extract_response(self, full_text: str, prompt: str) -> str:
        """Extract only the generated response from full text"""
        # Remove prompt from response
        if prompt in full_text:
            response = full_text.replace(prompt, "").strip()
        else:
            response = full_text.strip()

        # Clean up any remaining formatting
        response = response.replace("<|assistant|>", "").strip()
        response = response.replace("</s>", "").strip()

        return response


# ==================== Llama 3.2 Generator ====================

class Llama32Generator(BaseResponseGenerator):
    """Local Llama 3.2 3B generator for empathetic responses"""

    def __init__(self, config: ModelConfig):
        self.config = config
        self.model = None
        self.tokenizer = None
        self.streamer = None
        self._initialized = False

    def initialize(self):
        """Initialize Llama model and tokenizer"""
        if self._initialized:
            return

        logger.info("Initializing Llama 3.2 3B model...")

        try:
            # Check available memory
            if self.config.llama_device == "cuda":
                gpu = GPUtil.getGPUs()[0]
                logger.info(f"GPU Memory: {gpu.memoryUsed}MB / {gpu.memoryTotal}MB")

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.config.llama_model_name,
                trust_remote_code=True
            )

            # Set padding token
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load model with appropriate settings
            if self.config.llama_load_in_4bit:
                from transformers import BitsAndBytesConfig
                quantization_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_compute_dtype=self.config.llama_torch_dtype,
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_quant_type="nf4"
                )
            elif self.config.llama_load_in_8bit:
                from transformers import BitsAndBytesConfig
                quantization_config = BitsAndBytesConfig(
                    load_in_8bit=True,
                    bnb_8bit_compute_dtype=self.config.llama_torch_dtype
                )
            else:
                quantization_config = None

            # Load model
            self.model = AutoModelForCausalLM.from_pretrained(
                self.config.llama_model_name,
                torch_dtype=self.config.llama_torch_dtype,
                device_map="auto" if self.config.llama_device == "cuda" else None,
                quantization_config=quantization_config,
                trust_remote_code=True,
                max_memory=self.config.llama_max_memory if self.config.llama_device == "cuda" else None
            )

            if self.config.llama_device == "cpu":
                self.model = self.model.to(self.config.llama_device)

            # Set to evaluation mode
            self.model.eval()

            self._initialized = True
            logger.info("Llama 3.2 3B model initialized successfully!")

        except Exception as e:
            logger.error(f"Failed to initialize Llama model: {e}")
            raise

    async def generate(self, prompt: str, config: ModelConfig) -> Tuple[str, GenerationMetrics]:
        """Generate response using Llama 3.2"""
        if not self._initialized:
            self.initialize()

        start_time = time.time()

        try:
            # Format prompt for Llama 3.2 Instruct
            formatted_prompt = self._format_prompt(prompt)

            # Tokenize
            inputs = self.tokenizer(
                formatted_prompt,
                return_tensors="pt"
                # truncation=True,
                # max_length=2048 - config.max_new_tokens
            ).to(self.config.llama_device)

            # if self.config.llama_device == "cuda":
            #     inputs = {k: v.to(self.config.llama_device) for k, v in inputs.items()}

            prompt_tokens = inputs['input_ids'].shape[1]

             # Use TextIteratorStreamer for true streaming
            self.streamer = TextStreamer(self.tokenizer, skip_prompt=True, skip_special_tokens=True)

            # Run generation in a separate thread so it doesn't block the event loop
            generation_kwargs = dict(
                inputs,
                streamer=self.streamer,
                max_new_tokens=config.max_new_tokens,
                temperature=config.temperature,
                top_p=config.top_p,
                do_sample=config.do_sample,
                repetition_penalty=config.repetition_penalty,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

            # Generate
            # Using asyncio.to_thread for the blocking model call
            await asyncio.to_thread(self.model.generate, **generation_kwargs)

            # with torch.no_grad():
            #     if config.use_streaming:
            #         # Streaming generation
            #         response = await self._generate_streaming(inputs, config)
            #     else:
            #         # Standard generation
            #         outputs = self.model.generate(
            #             **inputs,
            #             max_new_tokens=config.max_new_tokens,
            #             temperature=config.temperature,
            #             top_p=config.top_p,
            #             do_sample=config.do_sample,
            #             repetition_penalty=config.repetition_penalty,
            #             pad_token_id=self.tokenizer.pad_token_id,
            #             eos_token_id=self.tokenizer.eos_token_id
            #         )

            #         # Decode
            #         full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            #         response = self.extract_response(full_response, formatted_prompt)
            with torch.no_grad():
                outputs = await asyncio.to_thread(
                    self.model.generate,
                    **inputs,
                    max_new_tokens=config.max_new_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9
                )

            response = self.tokenizer.decode(outputs[0][prompt_tokens:], skip_special_tokens=True)

            # Calculate metrics
            completion_tokens = len(self.tokenizer.encode(response))
            generation_time = time.time() - start_time

            metrics = GenerationMetrics(
                model_used="llama-3.2-3b",
                generation_time=generation_time,
                token_count=prompt_tokens + completion_tokens,
                prompt_tokens=prompt_tokens,
                completion_tokens=completion_tokens
            )

            return response, metrics

        except Exception as e:
            logger.error(f"Error generating with Llama 3.2: {e}")
            raise

    def _format_prompt(self, prompt: str) -> str:
        """Format prompt for Llama 3.2 Instruct"""
        # Llama 3.2 Instruct format
        return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    async def _generate_streaming(self, inputs: Dict, config: ModelConfig) -> str:
        """Generate response with streaming"""
        # For now, using standard generation
        # Streaming can be implemented with TextIteratorStreamer
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=config.max_new_tokens,
            temperature=config.temperature,
            top_p=config.top_p,
            do_sample=config.do_sample,
            repetition_penalty=config.repetition_penalty,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id
        )

        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self.extract_response(full_response, inputs['input_ids'])

    async def health_check(self) -> bool:
        """Check if Llama generator is healthy"""
        try:
            if not self._initialized:
                self.initialize()

            # Simple generation test
            test_prompt = "Hello, how are you?"
            inputs = self.tokenizer(test_prompt, return_tensors="pt")
            if self.config.llama_device == "cuda":
                inputs = {k: v.to(self.config.llama_device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=10,
                    do_sample=False
                )

            return True

        except Exception as e:
            logger.error(f"Llama health check failed: {e}")
            return False


# ==================== GPT-3.5 Fallback Generator ====================

class GPTGenerator(BaseResponseGenerator):
    """GPT-4.1 API generator as fallback"""

    def __init__(self, config: ModelConfig):
        self.config = config
        openai.api_key = config.gpt_api_key

    async def generate(self, prompt: str, config: ModelConfig) -> Tuple[str, GenerationMetrics]:
        """Generate response using GPT-4.1"""
        start_time = time.time()

        try:
            # Call OpenAI API
            response = await self._call_openai_api(prompt, config)

            # Extract response text
            response_text = response.choices[0].message.content

            # Calculate metrics
            generation_time = time.time() - start_time

            metrics = GenerationMetrics(
                model_used=config.gpt_model,
                generation_time=generation_time,
                token_count=response.usage.total_tokens,
                prompt_tokens=response.usage.prompt_tokens,
                completion_tokens=response.usage.completion_tokens,
                fallback_triggered=True
            )

            return response_text, metrics

        except Exception as e:
            logger.error(f"Error generating with GPT-3.5: {e}")
            raise

    async def _call_openai_api(self, prompt: str, config: ModelConfig) -> Dict:
        """Call OpenAI API with retry logic"""
        max_retries = 3
        retry_delay = 1.0

        for attempt in range(max_retries):
            try:
                response = await asyncio.to_thread(
                    openai.chat.completions.create,
                    model=config.gpt_model,
                    messages=[
                        {"role": "system", "content": "You are a highly empathetic, emotionally intelligent companion. Respond reflectively and with emotional presence."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=config.gpt_temperature,
                    max_tokens=config.gpt_max_tokens,
                    top_p=config.top_p

                )
                return response

            except openai.RateLimitError:
                if attempt < max_retries - 1:
                    await asyncio.sleep(retry_delay * (attempt + 1))
                else:
                    raise
            except (openai.APIError, openai.APIConnectionError, openai.Timeout) as e:
                logger.error(f"OpenAI API error: {e}")
                if attempt < max_retries - 1:
                    await asyncio.sleep(retry_delay * (attempt + 1))
                else:
                    raise
            except Exception as e:
                logger.error(f"Unexpected error: {e}")
                raise

    async def health_check(self) -> bool:
        """Check if GPT-3.5 API is accessible"""
        try:
            response = openai.chat.completions.create(
                model=self.config.gpt_model,
                messages=[{"role": "user", "content": "Hello"}],
                max_tokens=5
            )
            return True
        except Exception as e:
            logger.error(f"GPT-4.1-mini health check failed: {e}")
            return False


# ==================== Response Cache ====================

class ResponseCache:
    """Simple response cache for performance"""

    def __init__(self, max_size: int = 1000):
        self.cache = {}
        self.max_size = max_size
        self.access_count = {}

    def get(self, key: str) -> Optional[str]:
        """Get response from cache"""
        if key in self.cache:
            self.access_count[key] = self.access_count.get(key, 0) + 1
            return self.cache[key]
        return None

    def set(self, key: str, value: str):
        """Set response in cache"""
        if len(self.cache) >= self.max_size:
            # Remove least accessed item
            least_accessed = min(self.access_count.items(), key=lambda x: x[1])[0]
            del self.cache[least_accessed]
            del self.access_count[least_accessed]

        self.cache[key] = value
        self.access_count[key] = 1

    def clear(self):
        """Clear cache"""
        self.cache.clear()
        self.access_count.clear()


# ==================== Main Response Engine ====================

class KumoraResponseEngine:
    """
    Main response generation engine for Kumora
    Integrates emotion classification, prompt engineering, and model generation
    """

    def __init__(self, config: Optional[ModelConfig] = None):
        self.config = config or ModelConfig()

        # Initialize components
        logger.info("Initializing Kumora Response Engine...")

        # Existing components
        self.emotion_classifier = EmotionIntelligenceModule("kumora_emotion_model_final")
        self.context_manager = get_context_manager()  # This will use Redis if available, otherwise in-memory
        self.prompt_engineer = DynamicPromptEngineer()

        # Response generators
        self.llama_generator = Llama32Generator(self.config)
        self.gpt_generator = GPTGenerator(self.config)

        # Response cache
        self.cache = ResponseCache(self.config.max_cache_size) if self.config.cache_responses else None

        # Metrics
        self.total_requests = 0
        self.llama_success = 0
        self.fallback_count = 0

        logger.info("Kumora Response Engine initialized!")

    async def generate_response(self,
                              user_message: str,
                              user_id: str,
                              session_id: str,
                              use_fallback: bool = False) -> Dict[str, Any]:
        """
        Generate empathetic response for user message

        Args:
            user_message: User's input message
            user_id: User identifier
            session_id: Session identifier
            use_fallback: Force use of fallback model

        Returns:
            Dict containing response and metadata
        """
        self.total_requests += 1
        start_time = time.time()

        try:
            # Step 1: Analyze emotions
            logger.info("Analyzing emotions...")
            emotion_analysis = self.emotion_classifier.analyze_emotions(user_message)

            # Step 2: Get user context
            logger.info("Retrieving user context...")
            user_context = self._get_user_context(user_id, session_id)

            # Step 3: Create emotional context
            emotional_context = EmotionalContext(
                primary_emotion=emotion_analysis['primary_emotion'],
                detected_emotions=emotion_analysis['detected_emotions'],
                intensity=emotion_analysis['emotional_intensity'],
                valence=emotion_analysis['emotional_valence'],
                confidence=emotion_analysis.get('confidence', 0.8)
            )

            # Step 4: Determine prompt configuration
            prompt_config = self._determine_prompt_config(emotional_context, user_context)

            # Step 5: Generate dynamic prompt
            logger.info("Generating dynamic prompt...")
            prompt_result = self.prompt_engineer.generate_prompt(
                message=user_message,
                emotional_context=emotional_context,
                user_context=user_context,
                config=prompt_config
            )

            # Step 6: Check cache
            if self.cache and not use_fallback:
                cache_key = self._generate_cache_key(user_message, emotional_context)
                cached_response = self.cache.get(cache_key)
                if cached_response:
                    logger.info("Cache hit!")
                    return {
                        'response': cached_response,
                        'metadata': {
                            'cached': True,
                            'emotion_analysis': emotion_analysis,
                            'generation_time': 0.0
                        }
                    }

            # Step 7: Generate response
            response_text, metrics = await self._generate_with_model(
                prompt_result['system_prompt'],
                use_fallback
            )

            # Step 8: Post-process response
            final_response = self._post_process_response(
                response_text,
                emotional_context,
                user_context
            )

            # Step 9: Update context
            self._update_context(user_id, session_id, emotional_context, final_response)

            # Step 10: Cache if successful
            if self.cache and not use_fallback and metrics.model_used == "llama-3.2-3b":
                self.cache.set(cache_key, final_response)

            # Prepare result
            total_time = time.time() - start_time

            return {
                'response': final_response,
                'metadata': {
                    'emotion_analysis': emotion_analysis,
                    'support_type': prompt_result['metadata']['support_type'],
                    'model_used': metrics.model_used,
                    'generation_time': metrics.generation_time,
                    'total_time': total_time,
                    'cached': False,
                    'fallback_triggered': metrics.fallback_triggered,
                    'prompt_tokens': metrics.prompt_tokens,
                    'completion_tokens': metrics.completion_tokens
                }
            }

        except Exception as e:
            logger.error(f"Error generating response: {e}")

            # Emergency fallback
            return {
                'response': "I hear you, and I want to help. Could you tell me a bit more about what you're experiencing?",
                'metadata': {
                    'error': str(e),
                    'fallback': 'emergency'
                }
            }

    async def _generate_with_model(self, prompt: str, use_fallback: bool) -> Tuple[str, GenerationMetrics]:
        """Generate response using appropriate model"""

        # Try Llama first (unless fallback requested)
        if not use_fallback:
            try:
                logger.info("Generating with Llama 3.2...")
                response, metrics = await asyncio.wait_for(
                    self.llama_generator.generate(prompt, self.config),
                    timeout=self.config.response_timeout
                )
                self.llama_success += 1
                return response, metrics

            except asyncio.TimeoutError:
                logger.warning("Llama generation timed out, using fallback...")
                metrics_fallback_reason = "timeout"
            except Exception as e:
                logger.warning(f"Llama generation failed: {e}, using fallback...")
                metrics_fallback_reason = str(e)
        else:
            metrics_fallback_reason = "requested"

        # Use GPT-4.1 fallback
        self.fallback_count += 1
        logger.info("Generating with GPT-4.1-mini fallback...")

        try:
            response, metrics = await self.gpt_generator.generate(prompt, self.config)
            metrics.fallback_reason = metrics_fallback_reason
            return response, metrics
        except Exception as e:
            logger.error(f"Fallback generation also failed: {e}")
            raise

    def _get_user_context(self, user_id: str, session_id: str) -> UserContext:
        """Get or create user context"""
        try:
            # Get comprehensive context from context manager
            context = self.context_manager.get_comprehensive_context(user_id, session_id)

            # Convert to UserContext object
            return UserContext(
                user_id=user_id,
                active_goals=context.get('active_goals', []),
                recent_topics=[topic['topic'] for topic in context.get('session', {}).get('topics', [])],
                emotional_trajectory=context.get('emotional_trajectory', 'stable'),
                effective_strategies=[s['name'] for s in context.get('recommended_coping', [])],
                preferences=context.get('personalization', {}).get('preferences', {}),
                session_number=context.get('user_profile', {}).get('total_sessions', 1)
            )
        except:
            # Return minimal context if error
            return UserContext(user_id=user_id)

    def _determine_prompt_config(self, emotional_context: EmotionalContext,
                               user_context: UserContext) -> PromptConfig:
        """
        Determine optimal prompt configuration based on a holistic view of the
        emotional and user context.
        """
        # Base configuration
        config = PromptConfig()

        # 1. Determine Empathy Level
        # This logic considers intensity, but also the user relationship depth.
        relationship_depth = user_context.get_relationship_depth()
        if emotional_context.intensity > 0.8:
            config.empathy_level = EmpathyLevel.HIGH
        elif emotional_context.intensity > 0.5 and relationship_depth != "new":
            config.empathy_level = EmpathyLevel.MEDIUM
        else:
            # For new users or low intensity, let the calibrator decide.
            config.empathy_level = EmpathyLevel.ADAPTIVE

        # Override with user preference if it exists
        if user_context.preferences.get("empathy_preference"):
            pref = user_context.preferences["empathy_preference"]
            if pref == "high": config.empathy_level = EmpathyLevel.HIGH
            elif pref == "medium": config.empathy_level = EmpathyLevel.MEDIUM
            elif pref == "low": config.empathy_level = EmpathyLevel.LOW

        # 2. Determine Response Style
        # This logic is now more dynamic, pulling from emotion modifiers and trajectory.
        primary_emotion = emotional_context.primary_emotion
        emotion_mods = EMOTION_MODIFIERS.get(primary_emotion, {})
        tone_adjustments = emotion_mods.get('tone_adjustments', [])

        if 'encouraging' in tone_adjustments or 'upbeat' in tone_adjustments:
            config.response_style = ResponseStyle.ENCOURAGING
        elif 'calm' in tone_adjustments or 'reassuring' in tone_adjustments or 'grounding' in tone_adjustments:
            config.response_style = ResponseStyle.GENTLE
        elif user_context.emotional_trajectory == 'declining':
            config.response_style = ResponseStyle.GENTLE
        elif user_context.emotional_trajectory == 'improving':
            config.response_style = ResponseStyle.ENCOURAGING
        else:
            # Default to a warm and inviting style.
            config.response_style = ResponseStyle.WARM

        # 3. Determine when to use Chain-of-Thought (CoT)
        # CoT is used for complex, sensitive, or critical situations.
        use_cot = any([
            emotional_context.get_emotion_category() == "crisis",
            emotional_context.intensity > 0.8, # Very high intensity requires careful thought
            user_context.emotional_trajectory == 'declining', # User is struggling, needs thoughtful response
            len(emotional_context.detected_emotions) > 3, # Emotionally complex situation
        ])
        config.use_chain_of_thought = use_cot

        # 4. Determine when to include few-shot examples
        # Examples help guide the model in nuanced or high-stakes interactions.
        include_ex = any([
            emotional_context.intensity > 0.7,
            emotional_context.get_emotion_category() in ["crisis", "growth"],
            relationship_depth == "deep" # Guide the model on personalized tone for established users
        ])
        config.include_examples = include_ex

        return config

    def _post_process_response(self, response: str,
                             emotional_context: EmotionalContext,
                             user_context: UserContext) -> str:
        """Post-process generated response"""

        # Ensure response ends properly
        if response and not response[-1] in '.!?':
            response += '.'

        # Add user name if preferred and appropriate
        if user_context.preferences.get('use_name') and user_context.preferences.get('name'):
            name = user_context.preferences['name']
            # Add name at beginning for high-intensity emotions
            if emotional_context.intensity > 0.7 and not name.lower() in response.lower():
                response = f"{name}, {response[0].lower()}{response[1:]}"

        # Ensure minimum length for validation
        if len(response.split()) < 20 and emotional_context.get_emotion_category() != "general":
            response += " I'm here to listen and support you through this."

        return response

    def _update_context(self, user_id: str, session_id: str,
                       emotional_context: EmotionalContext, response: str):
        """Update context with interaction"""
        try:
            # Update emotional state in context
            emotional_state = EmotionalState(
                primary_emotion=emotional_context.primary_emotion,
                detected_emotions=emotional_context.detected_emotions,
                intensity=emotional_context.intensity,
                valence=emotional_context.valence,
                confidence=emotional_context.confidence
            )

            self.context_manager.session.update_emotional_state(session_id, emotional_state)

        except Exception as e:
            logger.warning(f"Failed to update context: {e}")

    def _generate_cache_key(self, message: str, emotional_context: EmotionalContext) -> str:
        """Generate cache key for response"""
        import hashlib

        key_components = [
            message[:100],  # First 100 chars
            emotional_context.primary_emotion,
            str(emotional_context.intensity),
            emotional_context.valence
        ]

        key_string = "|".join(key_components)
        return hashlib.md5(key_string.encode()).hexdigest()

    async def health_check(self) -> Dict[str, Any]:
        """Check health of all components"""
        health_status = {
            'status': 'healthy',
            'components': {},
            'metrics': {
                'total_requests': self.total_requests,
                'llama_success_rate': self.llama_success / max(self.total_requests, 1),
                'fallback_rate': self.fallback_count / max(self.total_requests, 1)
            }
        }

        # Check Llama
        try:
            llama_healthy = await self.llama_generator.health_check()
            health_status['components']['llama_3.2'] = 'healthy' if llama_healthy else 'unhealthy'
        except Exception as e:
            health_status['components']['llama_3.2'] = f'error: {str(e)}'
            health_status['status'] = 'degraded'

        # Check GPT-4.1-mini
        try:
            gpt_healthy = await self.gpt_generator.health_check()
            health_status['components']['gpt_3.5'] = 'healthy' if gpt_healthy else 'unhealthy'
        except Exception as e:
            health_status['components']['gpt_3.5'] = f'error: {str(e)}'
            if health_status['status'] == 'degraded':
                health_status['status'] = 'unhealthy'

        return health_status


# ==================== Standalone Functions ====================

async def initialize_kumora_engine(config: Optional[ModelConfig] = None) -> KumoraResponseEngine:
    """Initialize Kumora response engine with all components"""

    if config is None:
        # Default configuration
        config = ModelConfig()

        # Adjust based on available resources
        if torch.cuda.is_available():
            gpu = GPUtil.getGPUs()[0]
            available_memory = gpu.memoryTotal - gpu.memoryUsed

            if available_memory < 4000:  # Less than 4GB available
                logger.info("Limited GPU memory detected, using 4-bit quantization")
                config.llama_load_in_4bit = True
            elif available_memory < 6000:  # Less than 6GB available
                logger.info("Moderate GPU memory detected, using 8-bit quantization")
                config.llama_load_in_8bit = True

    engine = KumoraResponseEngine(config)

    # Pre-initialize Llama model
    logger.info("Pre-initializing Llama model...")
    engine.llama_generator.initialize()

    # Run health check
    health = await engine.health_check()
    logger.info(f"Engine health: {health}")

    return engine


# ==================== Usage Example ====================

if __name__ == "__main__":
    """Example usage of Kumora Response Engine"""

    # Initialize engine
    engine = await initialize_kumora_engine()

    # Test messages with different emotions
    test_messages = [
        {
            'message': "I'm feeling really anxious about my presentation tomorrow. I can't stop thinking about all the ways it could go wrong.",
            'user_id': 'test_user_001',
            'session_id': 'test_session_001'
        },
        {
            'message': "I finally got the promotion I've been working towards for years! I can't believe it actually happened!",
            'user_id': 'test_user_001',
            'session_id': 'test_session_001'
        },
        {
            'message': "I feel so alone. Nobody understands what I'm going through.",
            'user_id': 'test_user_002',
            'session_id': 'test_session_002'
        }
    ]

    for test in test_messages:
        print(f"\n{'='*60}")
        print(f"User: {test['message']}")
        print(f"{'='*60}")

        # Generate response
        result = await engine.generate_response(
            user_message=test['message'],
            user_id=test['user_id'],
            session_id=test['session_id']
        )

        print(f"\nKumora: {result['response']}")

        # Print metadata
        metadata = result['metadata']
        print(f"\nMetadata:")
        print(f"- Primary Emotion: {metadata['emotion_analysis']['primary_emotion']}")
        print(f"- Emotional Intensity: {metadata['emotion_analysis']['emotional_intensity']:.2f}")
        print(f"- Support Type: {metadata.get('support_type', 'unknown')}")
        print(f"- Model Used: {metadata.get('model_used', 'unknown')}")
        print(f"- Generation Time: {metadata.get('generation_time', 0):.2f}s")
        print(f"- Total Time: {metadata.get('total_time', 0):.2f}s")

    # Test fallback
    # print(f"\n{'='*60}")
    # print("Testing GPT-4.1-mini Fallback...")
    # print(f"{'='*60}")

    # fallback_result = engine.generate_response(
    #     user_message="I'm worried about my health.",
    #     user_id='test_user_003',
    #     session_id='test_session_003',
    #     use_fallback=True
    # )

    # print(f"\nKumora (Fallback): {fallback_result['response']}")
    # print(f"Model Used: {fallback_result['metadata'].get('model_used')}")


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



User: I'm feeling really anxious about my presentation tomorrow. I can't stop thinking about all the ways it could go wrong.
I want to start by acknowledging the unsettled energy you're carrying within yourself. 

It takes immense 
Kumora: I’m sensing a deep unsettled energy in you, like a storm quietly swirling beneath the surface, and my heart truly goes out to you for carrying that weight right now. It’s so brave of you to reach out and share this space with me, even when everything feels heavy or tangled inside. What you’re experiencing—whether it’s confusion, overwhelm, or something else—is a profoundly human moment, and it makes perfect sense that it feels difficult to navigate.

Sometimes our emotions can feel like waves that don’t quite settle, pulling us in different directions all at once. If you’re open to it, gently bringing your attention to the sensation of your feet on the ground or the rise and fall of your breath might offer a small anchor amid the restless energy. I’m here with you, not to rush or fix, but simply to hold space for whatever is present.

I wonder, if you feel comfortable, what it’s like for you to sit with these feelings right now? There’s no.

Metadata:
- Primar

beyond they're 
Kumora: I’m genuinely thrilled to hear about this accomplishment of yours! It’s such a wonderful moment to celebrate, and I want to acknowledge the effort and determination it took for you to get here. You’ve navigated through whatever challenges stood in your way, and that’s no small feat. It’s completely okay—more than okay, actually—to feel proud and to take this time to truly savor what you’ve achieved. 

I can imagine how meaningful this must be for you, and how it might be lighting up a sense of possibility or relief inside. What does this success feel like for you right now? How do you want to hold onto this feeling as you reflect on all the steps you took to make it happen?

We don’t often pause to honor our wins in the midst of life’s busyness, so this is your moment to soak it all in. Your accomplishment is a testament to your strength and resilience, and it’s worth celebrating fully—just as it is, right now. I.

Metadata:
- Primary Emotion: Productivity
- Emo

and 
Kumora: My heart truly goes out to you for reaching out in this moment. I can feel the depth of whatever you’re carrying right now—the weight of those emotions, heavy and raw. It’s perfectly okay to let yourself feel all of it, to allow tears as a gentle release if they come. Your feelings are valid and important, and I’m here to hold space for you without judgment or rush.

Sometimes, just naming the ache or the overwhelm inside can feel like a small, brave act of self-kindness. I want you to know that whatever you’re experiencing, it’s understandable and you’re not alone in it. If you want to share more about what’s on your heart or mind, I’m here to listen with warmth and care, without trying to fix or change a thing.

What feels most present for you right now, if you feel like sharing?

Metadata:
- Primary Emotion: Tearfulness
- Emotional Intensity: 0.92
- Support Type: validation
- Model Used: gpt-4.1-mini
- Generation Time: 3.01s
- Total Time: 33.08s


In [5]:
# Test messages with different emotions
test_messages = [
    {
        'message': "hello",
        'user_id': 'test_user_001',
        'session_id': 'test_session_001'
    },
    {
        'message': "I am doing great",
        'user_id': 'test_user_001',
        'session_id': 'test_session_001'
    },
    {
        'message': "I feel so alone. Nobody understands what I'm going through.",
        'user_id': 'test_user_002',
        'session_id': 'test_session_002'
    }
]

for test in test_messages:
    print(f"\n{'='*60}")
    print(f"User: {test['message']}")
    print(f"{'='*60}")

    # Generate response
    result = await engine.generate_response(
        user_message=test['message'],
        user_id=test['user_id'],
        session_id=test['session_id']
    )

    print(f"\nKumora: {result['response']}")

    # Print metadata
    metadata = result['metadata']
    print(f"\nMetadata:")
    print(f"- Primary Emotion: {metadata['emotion_analysis']['primary_emotion']}")
    print(f"- Emotional Intensity: {metadata['emotion_analysis']['emotional_intensity']:.2f}")
    print(f"- Support Type: {metadata.get('support_type', 'unknown')}")
    print(f"- Model Used: {metadata.get('model_used', 'unknown')}")
    print(f"- Generation Time: {metadata.get('generation_time', 0):.2f}s")
    print(f"- Total Time: {metadata.get('total_time', 0):.2f}s")


User: hello
want someone, you and to Welcome! know I'm that honored I'm that so I'm 


Kumora: Hello and welcome! It’s really wonderful to connect with you—how are you feeling as we start this conversation together? I'm here to listen and support you through this.

Metadata:
- Primary Emotion: Improved mood
- Emotional Intensity: 0.91
- Support Type: neutral
- Model Used: gpt-4.1-mini
- Generation Time: 1.03s
- Total Time: 31.18s

User: I am doing great
glad here 

ERROR:__main__:Error generating with Llama 3.2: Tensor on device cuda:0 is not on the expected device meta!


you've to 
Kumora: I’m truly touched by the courage it takes to reach out and share a piece of your inner world with me. It’s such a profound step, and I want you to know that my heart goes out to you as you navigate whatever emotions are swirling within. The very act of opening up, especially in a space that’s new and unfamiliar, speaks volumes about your strength and vitality. 

Whatever you’re carrying—joy, confusion, relief, or even pain—I see it all as part of your unique journey, rich with complexity and deeply human. Your feelings are valid and honored here, without any judgment or expectation. If you feel comfortable, I’d love to hear more about what this moment means to you. What sensations or thoughts are alive inside you right now as you share this? 

Let’s take this time to truly savor your experience together, embracing every layer of it with warmth and respect. You deserve to be seen, celebrated, and held gently in this space.

Metadata:
- Primary Emotion: Sexual drive
- 

just can 
Kumora: My heart truly goes out to you for stepping into this space and sharing whatever it is that’s weighing on you. I can feel the depth of your feelings just beneath the surface, even if the words aren’t fully there yet. It takes such courage to open up like this, and I want you to know that whatever emotions are swirling inside—whether they are sadness, confusion, overwhelm, or something else—they are completely valid and deserving of gentle attention.

Sometimes, emotions feel like a storm inside us, heavy and relentless, and it’s okay to let those tears come if they want to. Tears are not a sign of weakness; they are a release, a soft and natural language of the heart speaking its truth. If you feel like crying here, in this moment, please know you have my full permission to do so. I’m here with you, quietly holding space for every feeling, no matter how big or small.

I wonder what it feels like for you right now to carry this heav.

Metadata:
- Primary Emotion: Tearf

In [7]:
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
%pip install dotenv

In [1]:
%pip install GPUtil

In [2]:
!ls

drive  sample_data


In [5]:
%pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
%pip install aioredis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade aioredis